# Combining asprin and telingo for temporal preferences: A guidebook ⌚🫶🏼

This notebook presents possibilities for preference implementations in ASP, combining `asprin` and `telingo` to model temporal preferences. We model three different problems (dinner, monkey, travel) from two papers: 

1) Bienvenu, M., Fritz, C., & McIlraith, S. A. (2011). Specifying and computing preferred plans. Artificial Intelligence, 175(7-8), 1308-1345: https://www.sciencedirect.com/science/article/pii/S0004370210002109

2)  Son, T. C., & Pontelli, E. (2006). Planning with preferences using logic programming. Theory and Practice of Logic Programming, 6(5), 559-607. https://www.cambridge.org/core/journals/theory-and-practice-of-logic-programming/article/planning-with-preferences-using-logic-programming/441D04E11B5B0008367591DA813382A7

## Dinner Problem 🍝🧑🏻‍🍳🛒

Modelling after paper: **Bienvenu, M., Fritz, C., & McIlraith, S. A. (2011). Specifying and computing preferred plans. Artificial Intelligence, 175(7-8), 1308-1345:** https://www.sciencedirect.com/science/article/pii/S0004370210002109

Problem Framing in natural language:
    
*"Example 3.1 (The Dinner Domain). It’s dinner time and Claire is tired and hungry. Her goal is to be at home with her hunger sated. There are three possible ways for Claire to get food: she can cook something at home, order in take-out food, or go to a restaurant. To cook a meal, Claire needs to know how to make the meal and she must have the necessary ingredients, which might require a trip to the grocery store. She also needs a clean kitchen in which to prepare her meal. Ordering take-out is much simpler: she only has to order and eat the meal. Going to a restaurant requires getting to the restaurant, ordering, eating, and then returning home."*

The axiomatization of the dinner example is given on p. 1335 - 1336 and can be translated into ASP as follows: 

In [1]:
%%file dinner/instances.lp
% p. 1336

% MEALS

% meal(x) ≡ x = pizza ∨ x = tacos ∨ x = fajitas ∨ x = spaghetti
% ∨ x = sweetsourpork ∨ x = crêpes ∨ x = duck ∨ x = salad

meal(pizza). meal(tacos). meal(fajitas).meal(spaghetti).
meal(sweetsourpork). meal(crepes). meal(duck). meal(salad).

% TYPES OF MEALS

% vegetarian(x) ≡ x = salad
% italian(x) ≡ x = spaghetti ∨ x = pizza
% mexican(x) ≡ x = tacos ∨ x = fajitas
% french(x) ≡ x = crêpes ∨ x = duck
% chinese(x) ≡ x = sweetsourpork


vegetarian(salad). italian(spaghetti). italian(pizza). mexican(tacos).
mexican(fajitas). french(crepes). french(duck). chinese(sweetsourpork).


% LOCATIONS

% location(x) ≡ x = home ∨ x = store ∨ x = italianRest
% ∨ x = frenchRest ∨ x = chineseRest ∨ x = pizzaPlace
% (†) close(x, y) ≡ x = home ∧ y = italianRest

location(home). location(store). location(italianRest). location(frenchRest).
location(chineseRest). location(pizzaPlace). close(home, italianRest).


% TYPES OF RESTAURANTS

% takeOutRest(x) ≡ x = chineseRest ∨ x = pizzaPlace
% dineInRest(x) ≡ x = italianRest ∨ x = frenchRest

takeOutRest(chineseRest). takeOutRest(pizzaPlace).
dineInRest(italianRest). dineInRest(frenchRest).


% RESTAURANT OFFERINGS

% onMenu(x, y) ≡ y = italianRest ∧ (x = spaghetti ∨ x = pizza)
% ∨ y = frenchRest ∧ (x = crêpes ∨ x = duck)
% ∨ y = pizzaPlace ∧ x = pizza
% ∨ y = chineseRest ∧ x = sweetsourpork

onMenu(spaghetti, italianRest). onMenu(pizza, italianRest).
onMenu(crepes, frenchRest). onMenu(duck, frenchRest).
onMenu(pizza, pizzaPlace). onMenu(sweetsourpork, chineseRest).

% KNOWLEDGE OF RECIPES

% knowsHowToMake(x) ≡ x = crêpes ∨ x = spaghetti ∨ x = tacos
% ∨ x = fajitas ∨ x = salad

knowsHowToMake(crepes). knowsHowToMake(spaghetti). knowsHowToMake(tacos).
knowsHowToMake(fajitas). knowsHowToMake(salad).

Overwriting dinner/instances.lp


The action theory is represented in ASP as follows: 

In [16]:
%%file dinner/dinner.lp

% p. 1335
% ACTION PRECONDITION AXIOMS

% Poss(drive(x, y), s) ≡ location(x) ∧ location(y) ∧ x != y ∧ at(x, s)
action(drive(X,Y)) :- location(X), location(Y), X != Y.
prec(drive(X, Y), at(X), true) :- action(drive(X,Y)).

% Poss(walk(x, y), s) ≡ location(x) ∧ location(y) ∧ x != y ∧ at(x, s)
action(walk(X,Y)) :- location(X), location(Y), X != Y.
prec(walk(X, Y), at(X), true) :- action(walk(X,Y)).

% Poss(cook(x), s) ≡ meal(x) ∧ knowsHowToMake(x) ∧ at(home, s) ∧ hasIngredients(x, s) ∧ kitchenClean(s)
action(cook(X)) :- knowsHowToMake(X), meal(X).
prec(cook(X), at(home), true) :- action(cook(X)).
prec(cook(X), hasIngredients(X), true) :- action(cook(X)).
prec(cook(X), kitchenClean, true) :- action(cook(X)).


% Poss(eat(x), s) ≡ meal(x) ∧ (∃y(at(y, s) ∧ readyToEat(x, y, s)))
action(eat(X)) :- meal(X).


prec(eat(X), readyToEat(X,Y), true) : holds(at(Y)) :- action(eat(X)).

%Poss(buyIngredients(x), s) ≡ meal(x) ∧ ¬hasIngredients(x) ∧ at(store, s)
action(buyIngredients(X)) :- meal(X).
prec(buyIngredients(X), at(store), true) :- action(buyIngredients(X)).
prec(buyIngredients(X), holds(hasIngredients(X)), false) :- action(buyIngredients(X)).

% Poss(orderTakeout(x, y), s) ≡ meal(x) ∧ takeOutRest(y) ∧ onMenu(x, y) ∧ at(home, s)
action(orderTakeout(X,Y)) :- meal(X), takeOutRest(Y), onMenu(X,Y).
prec(orderTakeout(X,Y), at(home), true) :- action(orderTakeout(X,Y)).

% Poss(orderRestaurant(x, y), s) ≡ meal(x) ∧ dineInRest(y) ∧ onMenu(x, y) ∧ at(y, s)
action(orderRestaurant(X,Y)) :- meal(X), dineInRest(Y), onMenu(X,Y).
prec(orderRestaurant(X,Y), at(Y), true) :- action(orderRestaurant(X,Y)).

% Poss(cleanDishes, s) ≡ at(home, s)
action(cleanDishes).
prec(cleanDishes, at(home), true) :- action(cleanDishes).

:- prec(A,F,true), occurs(A), not prev(holds(F)).
:- prec(A,F,false), occurs(A), prev(holds(F)).

% EFFECTS

% a = drive(x, y) → at(y, do(a, s))
eff(drive(X,Y), at(Y), true) :- action(drive(X,Y)).

% a = drive(x, y) → ¬at(x, do(a, s))
eff(drive(X,Y), at(X), false) :- action(drive(X,Y)).

% a = walk(x, y) → at(y, do(a, s))
eff(walk(X,Y), at(Y), true) :- action(walk(X,Y)).

% a = walk(x, y) → ¬at(x, do(a, s))
eff(walk(X,Y), at(X), false) :- action(walk(X,Y)).

% isSnowing(s) ∧ a = walk(x, y) → cold(do(a, s))
condeffect(walk(X,Y), (), cold, true) :- action(walk(X,Y)).
condition(walk(X,Y), (), isSnowing, true) :- action(walk(X,Y)).

% a = cook(x) → readyToEat(x, home, do(a, s))
eff(cook(X), readyToEat(X, home), true) :- action(cook(X)).

% a = cook(x) → ¬hasIngredients(x, do(a, s))
eff(cook(X), hasIngredients(X), false) :- action(cook(X)).

% a = cook(x) → ¬kitchenClean(do(a, s))
eff(cook(X), kitchenClean, false) :- action(cook(X)).

% a = eat(x) → sated(do(a, s))
eff(eat(X), sated, true) :- action(eat(X)).

% at(y, s) ∧ a = eat(x) → ¬readyToEat(x, y, do(a, s))
condeffect(eat(X), (Y), readyToEat(X,Y), false) :- action(eat(X)), location(Y).
condition(eat(X), (Y), at(Y), true) :- location(Y), action(eat(X)).

% a = buyIngredients(x) → hasIngredients(x, do(a, s))
eff(buyIngredients(X), hasIngredients(X), true) :- action(buyIngredients(X)).

% a = orderTakeout(x, y) → readyToEat(x, home, do(a, s))
eff(orderTakeout(X,Y), readyToEat(X,home), true) :- action(orderTakeout(X,Y)).

% a = orderRestaurant(x, y) → readyToEat(x, y, do(a, s))
eff(orderRestaurant(X,Y), readyToEat(X,Y), true) :- action(orderRestaurant(X,Y)).

% a = cleanDishes → kitchenClean(do(a, s))
eff(cleanDishes, kitchenClean, true) :- action(cleanDishes).


% Initial States
% at(home, S0)
holds(at(home)) :- initially.

% kitchenClean(S0)
holds(kitchenClean) :-initially.

% hasIngredients(crêpes, S0)
holds(hasIngredients(crepes)) :- initially.


{holds(isSnowing)}.

% actions (these are all choices)
{occurs(X) : action(X)}1.


% Her goal is to be at home with her hunger sated
:- not eventually(holds(sated)).
:- finally, not holds(at(home)).

% Inertia
wnext(holds(F)) :- holds(F), not wnext(neg(F)).


% Update by effects
holds(F) :- occurs(A), eff(A, F, true).
neg(F) :- occurs(A), eff(A, F, false).

holds(F) :- occurs(A), condeffect(A, X, F, true), holds(G) : condition(A,X,G,true);
                                                  not holds(G) : condition(A,X,G,false).

neg(F) :- occurs(A), condeffect(A, X, F, false), holds(G) : condition(A,X,G,true);
                                                 not holds(G) : condition(A,X,G,false).

#external initially.
#external finally.
#external eventually(holds(sated)).
#external drive(X,Y) : location(X), location(Y).
#external isSnowing.
#external at(X) : location(X).
#external prev(holds(F)) : prec(A,F,true).
#external wnext(neg(F)) : prec(A,F,true).
#external condition(A,X,G,false) : action(A), location(X), at(G). % do we need this?

% show statements
#show show(occurs(X)) : action(X).
%#show show(holds(isSnowing)).

Overwriting dinner/dinner.lp


There are several answer sets satifying the goal, starting from horizon=2:

In [20]:
!clingo dinner/dinner.lp dinner/instances.lp  0 --output=reify | clingo - meta-telingo.lp -c horizon=2 0 --project

clingo version 5.4.0
Reading from - ...
Solving...
Answer: 1
(occurs(orderTakeout(sweetsourpork,chineseRest)),1) (occurs(eat(sweetsourpork)),2)
Answer: 2
(occurs(orderTakeout(pizza,pizzaPlace)),1) (occurs(eat(pizza)),2)
Answer: 3
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
SATISFIABLE

Models       : 3
Calls        : 1
Time         : 1.979s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 1.984s


Now we will look at the 11 preferences (soft constraints) as proposed by Bienvenu et al. (2011), presented in p. 1311-p-1312.

### Trajectory Property Formulas (TPF):

*"TPFs are used to describe properties of trajectories (sequences of actions and states). The TPFs f , r, and final(f ) are used to describe the static properties of states belonging to a trajectory, while the TPF occ(a) allows one to describe the types of actions that occur along a trajectory. These basic TPFs then serve as building blocks for creating more complex TPFs using the standard Boolean connectives, quantifiers, and temporal operators."* (p. 1311)

**1. `hasIngredients(spaghetti) ∧ knowsHowToMake(spaghetti) (P1)`**

→ *"The first TPF (P1) states that in the initial situation Claire has the ingredients and the know-how to cook spaghetti."* (p. 1311)

We translate this formula as:

In `dinner/auxiliary.lp`: 

`p1_aux :- initially, holds(hasIngredients(spaghetti)), knowsHowToMake(spaghetti).`

In [18]:
%%file dinner/pref.lp
#optimize(p1).

#preference(p1,more(cardinality)){
   and(initially, p1_aux)
   }.

Overwriting dinner/pref.lp


In [22]:
!clingo dinner/dinner.lp dinner/instances.lp dinner/auxiliary.lp dinner/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp dinner/pref.lp meta-telingo-extension.lp -c horizon=2 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND
Answer: 2
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 3
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 4
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 5
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 6
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 7
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 8
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 9
(occurs(eat(sweetsourpork)),2) (occurs(orderTakeout(sweetsourpork,chineseRest)),1)
OPTIMUM FOUND *
Answer: 10
(occurs(eat(sweetsourpork)),2) (occurs(orderTakeout(sweetsourpork,chineseRest)),1)
OPTIMUM FOUN

We see that the models are the same as before adding the preference since `holds(hasIngredients(spaghetti))` is never true initially. 

**2. `∃x(hasIngredients(x) & knowsHowToMake(x)) (P2)`**

→ *"(P2) is more general, expressing that in the initial situation Claire has the ingredients for something she knows how to make."* (p.1311)

We translate this formula as:

In `dinner/auxiliary.lp`: 

`p2_aux :- initially, holds(hasIngredients(X)), knowsHowToMake(X).`

In [23]:
%%file dinner/pref.lp
#optimize(p2).

#preference(p2,more(cardinality)){
   and(initially, p2_aux)
   }.

Overwriting dinner/pref.lp


In [24]:
!clingo dinner/dinner.lp dinner/instances.lp dinner/auxiliary.lp dinner/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp dinner/pref.lp meta-telingo-extension.lp -c horizon=2 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND
Answer: 2
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 3
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 4
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 5
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 6
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 7
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 8
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 9
(occurs(eat(sweetsourpork)),2) (occurs(orderTakeout(sweetsourpork,chineseRest)),1)
OPTIMUM FOUND *
Answer: 10
(occurs(eat(sweetsourpork)),2) (occurs(orderTakeout(sweetsourpork,chineseRest)),1)
OPTIMUM FOUN

Again, the answer sets are the same as without the preference since Claire always has the ingredients for something she knows how to make (crepes).

**3. `final(kitchenClean)(P3)`**

→ *"(P3) states that in the final situation the kitchen is clean."* (p.1311)

We translate this formula as: 

In `dinner/auxiliary.lp`: 

`p3_aux :- finally, holds(kitchenClean).`

In [25]:
%%file dinner/pref.lp
#optimize(p3).

#preference(p3,more(cardinality)){
   eventually(p3_aux)
   }.

Overwriting dinner/pref.lp


In [26]:
!clingo dinner/dinner.lp dinner/instances.lp dinner/auxiliary.lp dinner/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp dinner/pref.lp meta-telingo-extension.lp -c horizon=2 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND
Answer: 2
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 3
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 4
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 5
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 6
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 7
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 8
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 9
(occurs(eat(sweetsourpork)),2) (occurs(orderTakeout(sweetsourpork,chineseRest)),1)
OPTIMUM FOUND *
Answer: 10
(occurs(eat(sweetsourpork)),2) (occurs(orderTakeout(sweetsourpork,chineseRest)),1)
OPTIMUM FOUN

As opposed to before, `occurs(cook(crepes), 1)` and `occurs(eat(crepes), 2)` are not part of the answer sets anymore since cooking forces the kitchen to not be clean anymore, contradicting the preference. 

**4. `always(at(home)) (P4)`**

→ *"The TPF (P4) states that Claire remains at home throughout the trajectory."* (p.1311)

We translate this formula as: 

In `dinner/auxiliary.lp`: 

`always(p4_aux) :- initially, always(holds(at(home))).`

In [29]:
%%file dinner/pref.lp
#optimize(p4).

#preference(p4,more(cardinality)){
   always(p4_aux)
   }.

Overwriting dinner/pref.lp


For this preference, `horizon=2` is too short since this is not enough time for Claire to leave the house anyway. So we put `horizon=4` now.

Without preference: 

In [1]:
!clingo dinner/dinner.lp dinner/instances.lp  0 --output=reify | clingo - meta-telingo.lp -c horizon=4 20

clingo version 5.4.0
Reading from - ...
Solving...
Answer: 1
(occurs(orderTakeout(pizza,pizzaPlace)),1) (occurs(eat(pizza)),2) (occurs(walk(home,pizzaPlace)),3) (occurs(walk(pizzaPlace,home)),4)
Answer: 2
(occurs(orderTakeout(pizza,pizzaPlace)),1) (occurs(eat(pizza)),2) (occurs(walk(home,pizzaPlace)),3) (occurs(drive(pizzaPlace,home)),4)
Answer: 3
(occurs(orderTakeout(pizza,pizzaPlace)),1) (occurs(eat(pizza)),2) (occurs(walk(home,pizzaPlace)),3) (occurs(drive(pizzaPlace,home)),4)
Answer: 4
(occurs(orderTakeout(pizza,pizzaPlace)),1) (occurs(eat(pizza)),2) (occurs(walk(home,pizzaPlace)),3) (occurs(drive(pizzaPlace,home)),4)
Answer: 5
(occurs(orderTakeout(pizza,pizzaPlace)),1) (occurs(eat(pizza)),2) (occurs(walk(home,pizzaPlace)),3) (occurs(drive(pizzaPlace,home)),4)
Answer: 6
(occurs(orderTakeout(pizza,pizzaPlace)),1) (occurs(eat(pizza)),2) (occurs(walk(home,pizzaPlace)),3) (occurs(drive(pizzaPlace,home)),4)
Answer: 7
(occurs(orderTakeout(pizza,pizzaPlace)),1) (occurs(eat(pizza)),2) (occ

With preference: 

In [2]:
!clingo dinner/dinner.lp dinner/instances.lp dinner/auxiliary.lp dinner/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp dinner/pref.lp meta-telingo-extension.lp -c horizon=4 20 

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
assigned_weight(p11,0) assigned_weight(p10,4) assigned_weight(p13,4) (occurs(cleanDishes),2) (occurs(cook(crepes)),4) (occurs(eat(pizza)),3) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND
Answer: 2
assigned_weight(p11,0) assigned_weight(p10,4) assigned_weight(p13,4) (occurs(cook(crepes)),3) (occurs(eat(pizza)),2) (occurs(eat(crepes)),4) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 3
assigned_weight(p11,0) assigned_weight(p10,4) assigned_weight(p13,4) (occurs(cook(crepes)),3) (occurs(eat(pizza)),2) (occurs(eat(crepes)),4) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 4
assigned_weight(p11,0) assigned_weight(p10,4) assigned_weight(p13,4) (occurs(cook(crepes)),3) (occurs(eat(pizza)),2) (occurs(eat(crepes)),4) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 5
assigned_weight(p11,0) assigned_weight(p10,4) assigned_weight(p13,4) (occurs(cook(crepes)),3) (occurs

Here, `occurs(drive)` and `occurs(walk)` are never part of answer sets, Claire never leaves home. 

**5. `∃x eventually(occ(cook(x)))(P5)`**

**6. `∃x∃y eventually(occ(orderTakeout(x, y)))(P6)`**

**7. `∃x∃y eventually(occ(orderRestaurant(x, y)))(P7)`**

→ *"(P5)–(P7) state
respectively that at some point in time Claire cooks, orders in take-out, or orders a meal in a restaurant."* (p.1311)

These formula can be translated as: 

In `dinner/auxiliary.lp`: 
    
**`P5`**

`eventually(occurs(cook)) :- occurs(cook(_))`.

`p5_aux :- initially, eventually(occurs(cook(X)))`.


**`P6`**

`eventually(occurs(orderTakeout)) :- occurs(orderTakeout(X,Y))`.

`eventually(p6_aux) :- initially, eventually(occurs(orderTakeout))`.


**`P7`**

`eventually(occurs(orderRestaurant)) :- occurs(orderRestaurant(X,Y))`.

`eventually(p7_aux) :- initially, eventually(occurs(orderRestaurant))`.

In [32]:
%%file dinner/pref.lp

% (P5) x eventually(occ(cook(x)))

#optimize(p5).

#preference(p5,more(cardinality)){
   eventually(p5_aux)
   }.

Overwriting dinner/pref.lp


In [33]:
!clingo dinner/dinner.lp dinner/instances.lp dinner/auxiliary.lp dinner/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp dinner/pref.lp meta-telingo-extension.lp -c horizon=2 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
Answer: 2
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND
Answer: 3
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 4
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 5
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 6
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 7
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 8
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 9
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *

Models       : 9
  Optimum    : yes
  Optimal    : 8
Calls        : 5
Time         : 2.671s (Solving: 0.01s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 2.609s


→ Claire cooks something.

In [34]:
%%file dinner/pref.lp

% (P6) xy(eventually(occ(orderTakeout(x, y))))

#optimize(p6).

#preference(p6,more(cardinality)){
   eventually(p6_aux)
}.


Overwriting dinner/pref.lp


In [35]:
!clingo dinner/dinner.lp dinner/instances.lp dinner/auxiliary.lp dinner/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp dinner/pref.lp meta-telingo-extension.lp -c horizon=2 0 

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND
Answer: 2
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 3
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 4
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 5
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 6
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 7
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 8
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 9
(occurs(eat(sweetsourpork)),2) (occurs(orderTakeout(sweetsourpork,chineseRest)),1)
OPTIMUM FOUND *
Answer: 10
(occurs(eat(sweetsourpork)),2) (occurs(orderTakeout(sweetsourpork,chineseRest)),1)
OPTIMUM FOUN

→ Claire orders takeout.

In [36]:
%%file dinner/pref.lp


% (P7) xy(eventually(occ(orderRestaurant(x, y))))

#optimize(p7).

#preference(p7,more(cardinality)){
   eventually(p7_aux)
}.


Overwriting dinner/pref.lp


In [37]:
!clingo dinner/dinner.lp dinner/instances.lp dinner/auxiliary.lp dinner/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp dinner/pref.lp meta-telingo-extension.lp -c horizon=2 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND
Answer: 2
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 3
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 4
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 5
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 6
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 7
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 8
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 9
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 10
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 11
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 12
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace))

→ All models satisfy the goal since `horizon=2` is not enough time to order at the restaurant

In [38]:
!clingo dinner/dinner.lp dinner/instances.lp dinner/auxiliary.lp dinner/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp dinner/pref.lp meta-telingo-extension.lp -c horizon=4 20

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(cleanDishes),1) (occurs(cook(crepes)),4) (occurs(eat(pizza)),3) (occurs(orderTakeout(pizza,pizzaPlace)),2)
Answer: 2
(occurs(drive(italianRest,home)),4) (occurs(drive(home,italianRest)),1) (occurs(eat(spaghetti)),3) (occurs(orderRestaurant(spaghetti,italianRest)),2)
OPTIMUM FOUND
Answer: 3
(occurs(drive(frenchRest,home)),4) (occurs(drive(home,frenchRest)),1) (occurs(eat(crepes)),3) (occurs(orderRestaurant(crepes,frenchRest)),2)
OPTIMUM FOUND *
Answer: 4
(occurs(drive(frenchRest,home)),4) (occurs(drive(home,frenchRest)),1) (occurs(eat(crepes)),3) (occurs(orderRestaurant(crepes,frenchRest)),2)
OPTIMUM FOUND *
Answer: 5
(occurs(drive(frenchRest,home)),4) (occurs(drive(home,frenchRest)),1) (occurs(eat(crepes)),3) (occurs(orderRestaurant(crepes,frenchRest)),2)
OPTIMUM FOUND *
Answer: 6
(occurs(drive(frenchRest,home)),4) (occurs(drive(home,frenchRest)),1) (occurs(eat(crepes)),3) (occurs(orderRestaurant(crepes,frenchRest)),2

→ With `horizon=4`, Claire orders at the restaurant.

**8. `always(¬(∃x∃y occ(drive(x, y)∧ isSnowing ))) (P8)`**

→ *"The TPF (P8) states that at no point does Claire drive while it is snowing"* (p.1311)

We can translate this preference as: 

In `dinner/auxiliary.lp`: 
    
`occurs(drive) :- occurs(drive(X,Y)).`

`always(p8_aux) :- initially, always(negation(and(occurs(drive), holds(isSnowing)))).`


In [39]:
%%file dinner/pref.lp

#optimize(p8).

#preference(p8,more(cardinality)){
   always(p8_aux)
}.

Overwriting dinner/pref.lp


Without preference: 

In [40]:
!clingo dinner/dinner_snow.lp dinner/instances.lp  0 --output=reify | clingo - meta-telingo.lp -c horizon=4 20

clingo version 5.4.0
Reading from - ...
Solving...
Answer: 1
(occurs(orderTakeout(pizza,pizzaPlace)),1) (occurs(eat(pizza)),2) (occurs(walk(home,pizzaPlace)),3) (occurs(walk(pizzaPlace,home)),4)
Answer: 2
(occurs(orderTakeout(pizza,pizzaPlace)),1) (occurs(eat(pizza)),2) (occurs(walk(home,pizzaPlace)),3) (occurs(drive(pizzaPlace,home)),4)
Answer: 3
(holds(isSnowing),4) (occurs(orderTakeout(pizza,pizzaPlace)),1) (occurs(eat(pizza)),2) (occurs(walk(home,pizzaPlace)),3) (occurs(drive(pizzaPlace,home)),4)
Answer: 4
(holds(isSnowing),4) (holds(isSnowing),3) (occurs(orderTakeout(pizza,pizzaPlace)),1) (occurs(eat(pizza)),2) (occurs(walk(home,pizzaPlace)),3) (occurs(drive(pizzaPlace,home)),4)
Answer: 5
(holds(isSnowing),4) (holds(isSnowing),3) (holds(isSnowing),2) (occurs(orderTakeout(pizza,pizzaPlace)),1) (occurs(eat(pizza)),2) (occurs(walk(home,pizzaPlace)),3) (occurs(drive(pizzaPlace,home)),4)
Answer: 6
(holds(isSnowing),4) (holds(isSnowing),3) (holds(isSnowing),2) (holds(isSnowing),1) (occu

→ Claire drives in the snow, e.g.:

`(holds(isSnowing),4) (occurs(orderTakeout(pizza,pizzaPlace)),1) (occurs(eat(pizza)),2) (occurs(walk(home,pizzaPlace)),3) (occurs(drive(pizzaPlace,home)),4)`

With preference: 

In [41]:
!clingo dinner/dinner_snow.lp dinner/instances.lp dinner/auxiliary.lp dinner/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp dinner/pref.lp meta-telingo-extension.lp -c horizon=4 20

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(cleanDishes),1) (occurs(cook(crepes)),4) (occurs(eat(pizza)),3) (occurs(orderTakeout(pizza,pizzaPlace)),2)
OPTIMUM FOUND
Answer: 2
(occurs(drive(home,pizzaPlace)),1) (occurs(walk(pizzaPlace,home)),2) (occurs(cook(crepes)),3) (occurs(eat(crepes)),4)
OPTIMUM FOUND *
Answer: 3
(holds(isSnowing),4) (occurs(drive(home,pizzaPlace)),1) (occurs(walk(pizzaPlace,home)),2) (occurs(cook(crepes)),3) (occurs(eat(crepes)),4)
OPTIMUM FOUND *
Answer: 4
(holds(isSnowing),4) (holds(isSnowing),3) (occurs(drive(home,pizzaPlace)),1) (occurs(walk(pizzaPlace,home)),2) (occurs(cook(crepes)),3) (occurs(eat(crepes)),4)
OPTIMUM FOUND *
Answer: 5
(occurs(drive(home,pizzaPlace)),1) (occurs(walk(pizzaPlace,home)),2) (occurs(cook(crepes)),3) (occurs(eat(crepes)),4)
OPTIMUM FOUND *
Answer: 6
(holds(isSnowing),4) (occurs(drive(home,pizzaPlace)),1) (occurs(walk(pizzaPlace,home)),2) (occurs(cook(crepes)),3) (occurs(eat(crepes)),4)
OPTIMUM FOUND *
Answer

→ Claire never drives in the snow.

**9. `always(¬(∃x(occ(eat(x))∧ chinese(x))))(P9)`**

→ *"Finally (P9) tells us that Claire never eats any Chinese food."* (p.1311)

In `dinner/auxiliary.lp`: 
    
`occurs(eatChinese) :- occurs(eat(X)), chinese(X).`

`always(p9_aux) :- initially, always(negation(occurs(eatChinese))).`


In [42]:
%%file dinner/pref.lp

#optimize(p9).

#preference(p9,more(cardinality)){
   always(p9_aux)
}.

Overwriting dinner/pref.lp


In [43]:
!clingo dinner/dinner.lp dinner/instances.lp dinner/auxiliary.lp dinner/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp dinner/pref.lp meta-telingo-extension.lp -c horizon=2 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND
Answer: 2
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 3
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 4
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 5
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 6
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 7
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 8
(occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 9
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 10
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 11
(occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTI

→ She never eats chinese food

### Atomic Preference Formula (APF):

*"An atomic preference formula expresses a preference over alternatives. Each of the alternatives in the APF is annotated with a value from a totally ordered set V which describes how far that alternative is from the ideal. The lower the value, the closer to the ideal, the more satisfied the user. In what follows, we let V = [0, 1] for parsimony, but we could just as easily choose a strictly qualitative set like {best < good < indifferent < bad < worst} [...] To reiterate, an atomic preference formula represents a preference over alternatives ϕi . We wish to satisfy the TPF ϕi with the lowest index i. Consequently, if Claire eats pizza and crêpes, this is no better nor  worse with respect to (P10) than situations in which Claire eats only pizza, and it is strictly better than situations in which she just eats crêpes. Note that there is always implicitly one last option, which is to satisfy none of the ϕi , and this option is the least preferred"* (p. 1312)

In ASP, we can translate these APFs by minimizing weight functions.

**10. `occ'(eat(spaghetti) [0] >> occ'(eat(pizza)[0.4] >> occ'(eat(crêpes))[0.5] (P10)`**

 → *"From the values that Claire assigned to the various options, we can see that she has a strong preference for spaghetti but finds pizza and crêpes about equally appealing."*(p. 1312)

We can translate this as: 

In [44]:
%%file dinner/pref.lp
#optimize(p10).

assigned_weight(P, W) :- assigned_weight(P, X, W), #false: assigned_weight(P, Y, Z), Y < X.

#preference(p10, less(weight)) {
    W :: assigned_weight(p10, W)
}.
    
assigned_weight(p10, 1, 0) :- eventually(occurs(eat(spaghetti))).
assigned_weight(p10, 2, 4) :- eventually(occurs(eat(pizza))).
assigned_weight(p10, 3, 5) :-  eventually(occurs(eat(crepes))).
assigned_weight(p10, 4, 10) :-  and(negation(eventually(occurs(eat(crepes)))),
                            and(negation(eventually(occurs(eat(spaghetti)))), negation(eventually(occurs(eat(pizza)))))).
#show assigned_weight/2. 

Overwriting dinner/pref.lp


In [45]:
!clingo dinner/dinner.lp dinner/instances.lp dinner/auxiliary.lp dinner/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp dinner/pref.lp meta-telingo-extension.lp -c horizon=4 20

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
assigned_weight(p10,4) (occurs(drive(italianRest,home)),3) (occurs(drive(home,italianRest)),2) (occurs(eat(pizza)),4) (occurs(orderTakeout(pizza,pizzaPlace)),1)
Answer: 2
assigned_weight(p10,0) (occurs(walk(italianRest,home)),4) (occurs(walk(home,italianRest)),1) (occurs(eat(spaghetti)),3) (occurs(orderRestaurant(spaghetti,italianRest)),2)
OPTIMUM FOUND
Answer: 3
assigned_weight(p10,0) (occurs(walk(italianRest,home)),4) (occurs(walk(home,italianRest)),1) (occurs(eat(spaghetti)),3) (occurs(orderRestaurant(spaghetti,italianRest)),2)
OPTIMUM FOUND *
Answer: 4
assigned_weight(p10,0) (occurs(walk(italianRest,home)),4) (occurs(walk(home,italianRest)),1) (occurs(eat(spaghetti)),3) (occurs(orderRestaurant(spaghetti,italianRest)),2)
OPTIMUM FOUND *
Answer: 5
assigned_weight(p10,0) (occurs(walk(italianRest,home)),4) (occurs(walk(home,italianRest)),1) (occurs(eat(spaghetti)),3) (occurs(orderRestaurant(spaghetti,italianRest)),2)
OPTIMUM 

 → Answer sets containing `occurs(eat(spaghetti))` are preferred.

What if eating spaghetti is never possible?

In [47]:
%%file dinner/no-spaghetti.lp
not eventually(occurs(eat(spaghetti))).

Overwriting dinner/no-spaghetti.lp


In [48]:
!clingo dinner/dinner.lp dinner/no-spaghetti.lp dinner/instances.lp dinner/auxiliary.lp dinner/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp dinner/pref.lp meta-telingo-extension.lp -c horizon=4 20

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
assigned_weight(p10,4) (occurs(cook(crepes)),4) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND
Answer: 2
assigned_weight(p10,4) (occurs(walk(chineseRest,home)),2) (occurs(walk(home,chineseRest)),1) (occurs(eat(pizza)),4) (occurs(orderTakeout(pizza,pizzaPlace)),3)
OPTIMUM FOUND *
Answer: 3
assigned_weight(p10,4) (occurs(walk(chineseRest,home)),2) (occurs(walk(home,chineseRest)),1) (occurs(eat(pizza)),4) (occurs(orderTakeout(pizza,pizzaPlace)),3)
OPTIMUM FOUND *
Answer: 4
assigned_weight(p10,4) (occurs(walk(chineseRest,home)),2) (occurs(walk(home,chineseRest)),1) (occurs(eat(pizza)),4) (occurs(orderTakeout(pizza,pizzaPlace)),3)
OPTIMUM FOUND *
Answer: 5
assigned_weight(p10,4) (occurs(walk(chineseRest,home)),2) (occurs(walk(home,chineseRest)),1) (occurs(eat(pizza)),4) (occurs(orderTakeout(pizza,pizzaPlace)),3)
OPTIMUM FOUND *
Answer: 6
assigned_weight(p10,4) (occurs(walk(chineseRest,home)),2) (oc

 → If `occurs(eat(spaghetti))` is not possible, the next preferred answer sets contain `occurs(eat(pizza))`, and so on. 

**11. `P6[0] >> P5 ∧ P4[0.2] >> P7[0.7] >> P5 ∧ ¬P4[0.9]` (P11)**

 → *"This preference tells us that Claire’s first choice is take-out, followed by cooking if it doesn’t involve going out to get groceries, followed by going to a restaurant, and lastly cooking when it requires leaving her home. We can see here that Claire really prefers options that don’t involve going out."*(p. 1312)

We can translate this preference as: 

In [49]:
%%file dinner/pref.lp
#optimize(p11).

assigned_weight(P, W) :- assigned_weight(P, X, W), #false: assigned_weight(P, Y, Z), Y < X.

#preference(p11, less(weight)) {
    W :: assigned_weight(p11, W)
}.
    
assigned_weight(p11, 1, 0) :- eventually(p6_aux).
assigned_weight(p11, 2, 2) :- and(eventually(p5_aux), always(p4_aux)).
assigned_weight(p11, 3, 7) :- eventually(p7_aux).
assigned_weight(p11, 4, 9) :- and(eventually(p5_aux), negation(always(p4_aux))).

#show assigned_weight/2. 

Overwriting dinner/pref.lp


In [50]:
!clingo dinner/dinner.lp dinner/instances.lp dinner/auxiliary.lp dinner/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp dinner/pref.lp meta-telingo-extension.lp -c horizon=2 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
assigned_weight(p11,2) (occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
Answer: 2
assigned_weight(p11,0) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND
Answer: 3
assigned_weight(p11,0) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 4
assigned_weight(p11,0) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 5
assigned_weight(p11,0) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 6
assigned_weight(p11,0) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 7
assigned_weight(p11,0) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 8
assigned_weight(p11,0) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 9
assigned_weight(p11,0) (occurs(eat(pizza)),2) (occurs(

 → Answer sets containing `occurs(orderTakeout)` are preferred.

What happens if she can never order takeout?

In [52]:
%%file dinner/no-takeout.lp
not eventually(occurs(orderTakeout)).

Overwriting dinner/no-takeout.lp


In [53]:
!clingo dinner/dinner.lp dinner/no-takeout.lp dinner/instances.lp dinner/auxiliary.lp dinner/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp dinner/pref.lp meta-telingo-extension.lp -c horizon=2 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
assigned_weight(p11,2) (occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND
Answer: 2
assigned_weight(p11,2) (occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 3
assigned_weight(p11,2) (occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 4
assigned_weight(p11,2) (occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 5
assigned_weight(p11,2) (occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 6
assigned_weight(p11,2) (occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 7
assigned_weight(p11,2) (occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 8
assigned_weight(p11,2) (occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *

Models       : 8
  Optimum    : yes
  Optimal    : 8
Calls        : 4
Time         : 3.090s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 2.844s


 → If `occurs(orderTakeout)` is never available, the next possible preference, in this case, `eventually(occurs(cook))∧ always(holds(at(home)))` is satisfied, and so on.

### General Preference Formulas (GPF)

"**Definition 3.12** *(General Preference Satisfaction)*.  
Let $s$ be a situation and $\Phi$ be a general preference formula. Then $w_s(\Phi)$ is defined as follows:

- $w_S(\varphi_0[v_0] \gg \varphi_1[v_1] \gg \cdots \gg \varphi_n[v_n])$ is defined above.

- $$
  w_S(\gamma : \Psi) =
  \begin{cases}
    v_{\min} & \text{if } w_S(\gamma) = v_{\max} \\
    w_S(\Psi) & \text{otherwise}
  \end{cases}
  $$

- $$
  w_S(\Psi_1 \& \Psi_2 \& \cdots \& \Psi_n) = \max \{ w_S(\Psi_i) : 1 \le i \le n \}
  $$

- $$
  w_S(\Psi_1 \mid \Psi_2 \mid \cdots \mid \Psi_n) = \min \{ w_S(\Psi_i) : 1 \le i \le n \}
  $$

Observe that the semantics of our generalized Boolean connectives extends the semantics of their Boolean counterparts:
a conjunction $Ψ_1$&...& $Ψ_n$ is fully satisfied (i.e., has weight $v_{min}$
if all of the component preferences $Ψ_i$ are fully satisfied;
a disjunction $Ψ_1$ | ... | $Ψ_n$ is fully satisfied if at least one of the disjuncts $Ψ_i$ is fully satisfied; and a conditional preference
$γ$ : $Ψ$ is fully satisfied if either the condition $γ$ is false (i.e., has weight $v_{max}$) or the component preference formula $Ψ$ is
fully satisfied." (p. 1315)

**12. `P2 : P5 ∧ P4` (P12)**

 → "*(P12) states that if Claire initially has the ingredients for something she can make, then she prefers to stay in and cook.*"

We can translate this to: 

In [54]:
%%file dinner/pref.lp
#optimize(p12).

assigned_weight(P, W) :- assigned_weight(P, X, W), #false: assigned_weight(P, Y, Z), Y < X.

#preference(p12, less(weight)) {
     W :: assigned_weight(p12, W)
}.
    
assigned_weight(p12, 1, 0) :- if(and(p2_aux, initially), and(eventually(p5_aux), always(p4_aux))).
assigned_weight(p12, 2, 10) :- not if(and(p2_aux, initially), and(eventually(p5_aux), always(p4_aux))).

#show assigned_weight/2. 

Overwriting dinner/pref.lp


In [55]:
!clingo dinner/dinner.lp dinner/instances.lp dinner/auxiliary.lp dinner/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp dinner/pref.lp meta-telingo-extension.lp -c horizon=2 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
assigned_weight(p12,10) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
Answer: 2
assigned_weight(p12,0) (occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND
Answer: 3
assigned_weight(p12,0) (occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 4
assigned_weight(p12,0) (occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 5
assigned_weight(p12,0) (occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 6
assigned_weight(p12,0) (occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 7
assigned_weight(p12,0) (occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 8
assigned_weight(p12,0) (occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *
Answer: 9
assigned_weight(p12,0) (occurs(cook(crepes)),1) (occurs(eat(crepes)),2)
OPTIMUM FOUND *

Models       : 9
  Optimum    : yes
  Optimal    : 8
Calls        

→ Since Claire has the ingredients for crepes, she stays in and cooks at home. 

**13. `P10 & P11 ` (P13)**

→ "*(P13) maximizes the
satisfaction of both Claire’s food and time preferences ...*" (p. 1312)

This can be translated as: 

In [56]:
%%file dinner/pref.lp
#optimize(p13).

#preference(p13, less(weight)) {
    W :: assigned_weight(p13, W)
}.

assigned_weight(p13, W1) :- assigned_weight(p10, W0), assigned_weight(p11, W1), W0 <= W1.
assigned_weight(p13, W1) :- assigned_weight(p11, W0), assigned_weight(p10, W1), W0 <= W1.

    
assigned_weight(P, W) :- assigned_weight(P, X, W), #false: assigned_weight(P, Y, Z), Y < X.


assigned_weight(p10, 1, 0) :- eventually(occurs(eat(spaghetti))).
assigned_weight(p10, 2, 4) :- eventually(occurs(eat(pizza))).
assigned_weight(p10, 3, 5) :-  eventually(occurs(eat(crepes))).
assigned_weight(p10, 4, 10) :-  and(negation(eventually(occurs(eat(crepes)))),
                            and(negation(eventually(occurs(eat(spaghetti)))), negation(eventually(occurs(eat(pizza)))))).

assigned_weight(p11, 1, 0) :- eventually(p6_aux).
assigned_weight(p11, 2, 2) :- and(eventually(p5_aux), always(p4_aux)).
assigned_weight(p11, 3, 7) :- eventually(p7_aux).
assigned_weight(p11, 4, 9) :- and(eventually(p5_aux), negation(always(p4_aux))).

#show assigned_weight/2. 

Overwriting dinner/pref.lp


In [57]:
!clingo dinner/dinner.lp dinner/instances.lp dinner/auxiliary.lp dinner/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp dinner/pref.lp meta-telingo-extension.lp -c horizon=2 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
assigned_weight(p11,0) assigned_weight(p10,4) assigned_weight(p13,4) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND
Answer: 2
assigned_weight(p11,0) assigned_weight(p10,4) assigned_weight(p13,4) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 3
assigned_weight(p11,0) assigned_weight(p10,4) assigned_weight(p13,4) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 4
assigned_weight(p11,0) assigned_weight(p10,4) assigned_weight(p13,4) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 5
assigned_weight(p11,0) assigned_weight(p10,4) assigned_weight(p13,4) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 6
assigned_weight(p11,0) assigned_weight(p10,4) assigned_weight(p13,4) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
O

→ Answer sets containing `occurs(orderTakeout)` **and** `occurs(eat(pizza))` are
preferred, since they satisfy the time preference (P11) and achieve the highest possible satisfaction of the food preference (P10) (`eat(spaghetti)` is not available due to `horizon=2`). We also see that the overall score is 4 (`assigned_weight(p13,4)`) since we take the maximum value of both preferences (`assigned_weight(p10,4)`, (`assigned_weight(p11,0)`)) (see **Definition 3.12**).

**14. `P10 | P11 ` (P14)**

→  "*whereas (P14) says that she is content if either of the two were satisfied*" (p. 1312)

This can be translated as: 

In [37]:
%%file dinner/pref.lp
#optimize(p14).

#preference(p14, less(weight)) {
    W :: assigned_weight(p14, W)
}.

assigned_weight(p14, W0) :- assigned_weight(p10, W0), assigned_weight(p11, W1), W0 <= W1.
assigned_weight(p14, W0)  :- assigned_weight(p11, W0), assigned_weight(p10, W1), W0 <= W1.

    
assigned_weight(P, W) :- assigned_weight(P, X, W), #false: assigned_weight(P, Y, Z), Y < X.


assigned_weight(p10, 1, 0) :- eventually(occurs(eat(spaghetti))).
assigned_weight(p10, 2, 4) :- eventually(occurs(eat(pizza))).
assigned_weight(p10, 3, 5) :-  eventually(occurs(eat(crepes))).
assigned_weight(p10, 4, 10) :-  and(negation(eventually(occurs(eat(crepes)))),
                            and(negation(eventually(occurs(eat(spaghetti)))), negation(eventually(occurs(eat(pizza)))))).

assigned_weight(p11, 1, 0) :- eventually(p6_aux).
assigned_weight(p11, 2, 2) :- and(eventually(p5_aux), always(p4_aux)).
assigned_weight(p11, 3, 7) :- eventually(p7_aux).
assigned_weight(p11, 4, 9) :- and(eventually(p5_aux), negation(always(p4_aux))).

#show assigned_weight/2. 

Overwriting dinner/pref.lp


In [38]:
!clingo dinner/dinner.lp dinner/instances.lp dinner/auxiliary.lp dinner/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp dinner/pref.lp meta-telingo-extension.lp -c horizon=2 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
assigned_weight(p11,0) assigned_weight(p10,4) assigned_weight(p14,0) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND
Answer: 2
assigned_weight(p11,0) assigned_weight(p10,4) assigned_weight(p14,0) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 3
assigned_weight(p11,0) assigned_weight(p10,4) assigned_weight(p14,0) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 4
assigned_weight(p11,0) assigned_weight(p10,4) assigned_weight(p14,0) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 5
assigned_weight(p11,0) assigned_weight(p10,4) assigned_weight(p14,0) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
OPTIMUM FOUND *
Answer: 6
assigned_weight(p11,0) assigned_weight(p10,4) assigned_weight(p14,0) (occurs(eat(pizza)),2) (occurs(orderTakeout(pizza,pizzaPlace)),1)
O

→ Here, answer sets containing `occurs(orderTakeout)` **or** `occurs(eat(pizza))` are preferred. As opposed to before, where the overall preference score was 4, it is now 0 since we take the minimum value of both preferences (`assigned_weight(p10,4)`, (`assigned_weight(p11,0)`), (`assigned_weight(p14,0)`)) (see **Definition 3.12**)

## Travel Problem ✈️🚌🚗

The next planning problem is from the travel domain: An agent's goal is to get to school, with different possibilities of transport. 

Modelling after paper: **Son, T. C., & Pontelli, E. (2006). Planning with preferences using logic programming. Theory and Practice of Logic Programming, 6(5), 559-607.** https://www.cambridge.org/core/journals/theory-and-practice-of-logic-programming/article/planning-with-preferences-using-logic-programming/441D04E11B5B0008367591DA813382A7

The proposed actions with their effects and executabilty condition can be found in p. 563. They can be translated as follows: 

In [75]:
%%file travel/travel.lp 

% FLUENTS
% initially (at(home))
holds(at(home)) :- initially.

% initially (has money)
holds(has_money) :-initially.

% initially (available car(home))
holds(available_car(home)) :- initially.

holds(sCost(0)) :- initially.
holds(sCost(NC)) :-
    prev(holds(sCost(N))),
    occurs(A),
    cost(A, C),
    num(N), num(C), NC = N + C, num(NC).

wnext(neg(sCost(N))) :-
    holds(sCost(N)),
    wnext(holds(sCost(M))), N != M.


% ACTIONS WITH THEIR EXECUTABILITY CONDITIONS (ACTION PRECONDITION AXIOMS)

% walk(l1, l2) causes at(l2) if at(l1), road(l1, l2)
action(walk(X,Y)) :- road(X,Y).
prec(walk(X, Y), at(X), true) :- action(walk(X,Y)).

wnext(executable(walk)) :- holds(at(X)), action(walk(X,Y)).

% bus(l1, l2) causes at(l2) if at(l1), road(l1, l2)
action(bus(X,Y)) :- bus_station(X), bus_station(Y), road(X,Y).
prec(bus(X, Y), at(X), true) :- action(bus(X,Y)).

% bus(l1, l2) executable if has money
prec(bus(X, Y), has_money, true) :- action(bus(X,Y)).

wnext(executable(bus)) :- holds(at(X)), action(bus(X,Y)), holds(has_money).


% flight(l1, l2) causes at(l2) if at(l1), has_ticket(l1, l2)
% flight(l1, l2) executable if connected(l1, l2)
action(flight(X,Y)) :- connected(X,Y).
prec(flight(X, Y), at(X), true) :- action(flight(X,Y)).
prec(flight(X, Y), has_ticket(X, Y), true) :- action(flight(X,Y)).
wnext(executable(flight)) :- action(flight(X,Y)), holds(at(X)), holds(has_ticket(X, Y)).

% take_taxi(l1, l2) causes at(l2) if at(l1), road(l1, l2)
action(take_taxi(X,Y)) :- road(X,Y).
prec(take_taxi(X, Y), at(X), true) :- action(take_taxi(X,Y)).


% take_taxi(l1, l2) executable if available_taxi(l1)
prec(take_taxi(X, Y), available_taxi(X), true) :- action(take_taxi(X,Y)).

wnext(executable(take_taxi)) :- action(take_taxi(X,Y)), holds(available_taxi(X)), holds(at(X)).

% call_taxi(l) causes available_taxi(l) if has_money
action(call_taxi(X)) :- location(X).
prec(call_taxi(X), has_money, true) :- action(call_taxi(X)).
wnext(executable(call_taxi)) :- action(call_taxi(X)), holds(has_money).

% rent_car(l) causes available car(l) if has money
action(rent_car(X)) :- location(X).
prec(rent_car(X), has_money, true) :- action(rent_car(X)).
wnext(executable(rent_car)) :- action(rent_car(X)), holds(has_money).

% flight(l1, l2) executable if connected(l1, l2)
wnext(executable(flight)) :- action(flight(X, Y)), holds(connected(X,Y)).

% 
action(drive(X,Y)) :- road(X,Y).
prec(drive(X, Y), available_car(X), true) :- action(drive(X,Y)).
prec(drive(X, Y), at(X), true) :- action(drive(X,Y)).
wnext(executable(drive)) :- holds(available_car(X)), holds(at(X)), action(drive(X,Y)).


% buy ticket(l1, l2) executable if has_money
action(buy_ticket(X,Y)) :- airport(X), airport(Y), X != Y.
prec(buy_ticket(X, Y), has_money, true) :- action(buy_ticket(X,Y)).
wnext(executable(buy_ticket)) :- holds(has_money), action(buy_ticket(X,Y)).

    
action(buy_coffee).
prec(buy_coffee, has_money, true) :- action(buy_coffee).
prec(buy_coffee, at(coffeeshop), true) :- action(buy_coffee).
wnext(executable(buy_coffee)) :- holds(at(coffeeshop)), holds(has_money).

:- prec(A,F,true), occurs(A), not prev(holds(F)).
:- prec(A,F,false), occurs(A), prev(holds(F)).

%executable(A) :- prec(A,F,true), prev(holds(F)).
%executable(A) :- prec(A,F,false), not prev(holds(F)).

% ACTIONS WITH THEIR EFFECTS (EFFECTS)

% walk(l1, l2) causes at(l2) if at(l1), road(l1, l2)
eff(walk(X,Y), at(Y), true) :- action(walk(X,Y)).
eff(walk(X,Y), at(X), false) :- action(walk(X,Y)).

% bus(l1, l2) causes at(l2) if at(l1), road(l1, l2)
eff(bus(X,Y), at(Y), true) :- action(bus(X,Y)).
eff(bus(X,Y), at(X), false) :- action(bus(X,Y)).

% flight(l1, l2) causes at(l2) if at(l1), has ticket(l1, l2)
eff(flight(X,Y), at(Y), true) :- action(flight(X,Y)).
eff(flight(X,Y), at(X), false) :- action(flight(X,Y)).


% take_taxi(l1, l2) causes at(l2) if at(l1), road(l1, l2)
eff(take_taxi(X,Y), at(Y), true) :- action(take_taxi(X,Y)).
eff(take_taxi(X,Y), at(X), false) :- action(take_taxi(X,Y)).

% buy_ticket(l1, l2) causes has_ticket(l1, l2)
eff(buy_ticket(X,Y), has_ticket(X,Y), true) :- action(buy_ticket(X,Y)).

% call_taxi(l) causes available_taxi(l) if has money
eff(call_taxi(X), available_taxi(X), true) :- action(call_taxi(X)).

% rent_car(l) causes available_car(l) if has money
eff(rent_car(X), available_car(X), true) :- action(rent_car(X)).


% add action buy_coffee (page 567)
eff(buy_coffee, has_coffee, true) :- action(buy_coffee).

    
eff(drive(X,Y), at(Y), true) :- action(drive(X,Y)).
eff(drive(X,Y), at(X), false) :- action(drive(X,Y)).
eff(flight(X,Y), has_ticket(X,Y), false) :- action(flight(X,Y)).
eff(take_taxi(X,Y), available_taxi(X), false) :- action(take_taxi(X,Y)).
eff(drive(X,Y), available_car(X), false) :- action(drive(X,Y)).


% Update by effects
holds(F) :- occurs(A), eff(A, F, true).
neg(F) :- occurs(A), eff(A, F, false).

{ occurs(A) : action(A), A != stop, A != noop } 1 :- not holds(ended).

% INERTIA
wnext(holds(F)) :- holds(F), not wnext(neg(F)).


% GOAL
holds(goal_achieved) :-  holds(at(school)).

% for preferences time_coffee, cost_coffee, time_n_cost, time_and_cost, time_v_cost, time_or_cost:
%holds(goal_achieved) :- holds(at(school)), holds(has_coffee).
:- finally, not holds(goal_achieved).

% as soon as the goal is achieved, fire stop (once)
wnext(occurs(stop)) :- holds(goal_achieved), not holds(ended).
prec(stop, goal_achieved, true).   % stop executable if goal
wnext(executable(stop)) :- holds(goal_achieved), not holds(ended).


eff(stop, ended,     true).        % stop causes ended

% after ended holds, fire noop every step
wnext(occurs(noop)) :- holds(ended).
prec(noop, ended,    true).        % noop executable if ended
wnext(executable(noop)) :-  holds(ended).

eff(noop, ended, true).        % noop causes ended


#external initially.
#external finally.
#external occurs(stop).
#external wnext(occurs(noop)).
#external holds(ended).
#external eventually(holds(goal_achieved)).
#external eventually(holds(at(school))).
#external holds(has_coffee).
#external prev(holds(sCost(N))) : holds(sCost(N)).
#external wnext(holds(sCost(M))) : holds(sCost(M)).
#external prev(holds(sTime(N))) : holds(sTime(N)).
#external wnext(holds(sTime(M))) : holds(sTime(M)).
#external prev(holds(F)) : prec(A,F,true).
#external wnext(neg(F)) : prec(A,F,true).
#external at(X) : location(X).
#external wnext(neg(F)) : prec(A,F,true).
#external eventually(holds(has_coffee)).


#show show(occurs(X)) : action(X).


Overwriting travel/travel.lp


The goal can be achieved in one step: 

In [76]:
! clingo travel/travel.lp travel/instances.lp  0 --output=reify | clingo - meta-telingo.lp -c horizon=1 0 --project

clingo version 5.4.0
Reading from - ...
Solving...
Answer: 1
(occurs(bus(home,school)),1)
Answer: 2
(occurs(walk(home,school)),1)
Answer: 3
(occurs(drive(home,school)),1)
SATISFIABLE

Models       : 3
Calls        : 1
Time         : 0.896s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.891s


Increasing the number of horizon quickly increases the number of possibilities: 

In [62]:
! clingo travel/travel.lp travel/instances.lp  0 --output=reify | clingo - meta-telingo.lp -c horizon=2 0 --project

clingo version 5.4.0
Reading from - ...
Solving...
Answer: 1
(occurs(bus(coffeeshop,school)),2) (occurs(drive(home,coffeeshop)),1)
Answer: 2
(occurs(bus(coffeeshop,school)),2) (occurs(walk(home,coffeeshop)),1)
Answer: 3
(occurs(bus(coffeeshop,school)),2) (occurs(bus(home,coffeeshop)),1)
Answer: 4
(occurs(bus(home,school)),1)
Answer: 5
(occurs(walk(home,school)),1)
Answer: 6
(occurs(drive(home,school)),1)
Answer: 7
(occurs(bus(home,school)),2)
Answer: 8
(occurs(bus(home,school)),2) (occurs(rent_car(home)),1)
Answer: 9
(occurs(bus(home,school)),2) (occurs(buy_ticket(home,school)),1)
Answer: 10
(occurs(bus(home,school)),2) (occurs(buy_ticket(airport1,school)),1)
Answer: 11
(occurs(bus(home,school)),2) (occurs(buy_ticket(school,airport1)),1)
Answer: 12
(occurs(bus(home,school)),2) (occurs(buy_ticket(home,airport2)),1)
Answer: 13
(occurs(bus(home,school)),2) (occurs(buy_ticket(home,airport1)),1)
Answer: 14
(occurs(bus(home,school)),2) (occurs(buy_ticket(school,home)),1)
Answer: 15
(occurs(b

Thus, we now look at preferences as proposed by the authors. 

### Basic Desire Formula

**1. `eventually(occ(bus(home, school)) ∨ occ(taxi(home, school))).`**

→  "*... to express the fact that a user would like to take the taxi or the bus to go to school*" (p. 566)

We can translate this temporal formula straightforward as: 

In [64]:
%%file travel/pref.lp
#optimize(p1).

% (P1) eventually(occ(bus(home,school)) | occ(taxi(home,school))).-> p. 566.
#preference(p1,more(cardinality)){
    and(initially, eventually(or(occurs(bus(home,school)), occurs(take_taxi(home,school)))))
}.

Overwriting travel/pref.lp


In [65]:
!clingo travel/travel.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=2 0 

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(walk(home,school)),2) (occurs(rent_car(home)),1)
Answer: 2
(occurs(bus(home,school)),2) (occurs(buy_ticket(home,school)),1)
OPTIMUM FOUND
Answer: 3
(occurs(take_taxi(home,school)),2) (occurs(call_taxi(home)),1)
OPTIMUM FOUND *
Answer: 4
(occurs(bus(home,school)),2) (occurs(call_taxi(home)),1)
OPTIMUM FOUND *
Answer: 5
(occurs(bus(home,school)),2) (occurs(call_taxi(airport1)),1)
OPTIMUM FOUND *
Answer: 6
(occurs(bus(home,school)),2) (occurs(call_taxi(school)),1)
OPTIMUM FOUND *
Answer: 7
(occurs(bus(home,school)),2) (occurs(call_taxi(coffeeshop)),1)
OPTIMUM FOUND *
Answer: 8
(occurs(bus(home,school)),2) (occurs(call_taxi(airport2)),1)
OPTIMUM FOUND *
Answer: 9
(occurs(bus(home,school)),2) (occurs(buy_ticket(airport1,airport2)),1)
OPTIMUM FOUND *
Answer: 10
(occurs(bus(home,school)),2) (occurs(buy_ticket(school,home)),1)
OPTIMUM FOUND *
Answer: 11
(occurs(bus(home,school)),2) (occurs(rent_car(coffeeshop)),1)
OPTIMUM FOU

Now, the agent either takes the bus or the taxi to get from home to school at some point of the trajectory. 

**2. `always(¬occ(call_taxi(home)))`**

→ "*If the user’s desire is not to call a taxi*" (p. 566)

Preference with asprin: 

In [66]:
%%file travel/pref.lp
#optimize(p2).

% (P2) always(¬occ(call_taxi(home))) -> p. 566.
#preference(p2,more(cardinality)){
    and(initially, always(negation(occurs(call_taxi(home)))))
}.

Overwriting travel/pref.lp


In [67]:
!clingo travel/travel.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=2 0 

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(walk(home,school)),2) (occurs(rent_car(home)),1)
OPTIMUM FOUND
Answer: 2
(occurs(walk(home,school)),2) (occurs(buy_ticket(airport1,airport2)),1)
OPTIMUM FOUND *
Answer: 3
(occurs(walk(home,school)),2) (occurs(rent_car(coffeeshop)),1)
OPTIMUM FOUND *
Answer: 4
(occurs(walk(home,school)),2) (occurs(buy_ticket(school,home)),1)
OPTIMUM FOUND *
Answer: 5
(occurs(walk(home,school)),2) (occurs(rent_car(school)),1)
OPTIMUM FOUND *
Answer: 6
(occurs(walk(home,school)),2) (occurs(call_taxi(coffeeshop)),1)
OPTIMUM FOUND *
Answer: 7
(occurs(walk(home,school)),2) (occurs(call_taxi(airport1)),1)
OPTIMUM FOUND *
Answer: 8
(occurs(walk(home,school)),2) (occurs(call_taxi(school)),1)
OPTIMUM FOUND *
Answer: 9
(occurs(walk(home,school)),2) (occurs(rent_car(airport1)),1)
OPTIMUM FOUND *
Answer: 10
(occurs(walk(home,school)),2) (occurs(call_taxi(airport2)),1)
OPTIMUM FOUND *
Answer: 11
(occurs(walk(home,school)),2)
OPTIMUM FOUND *
Answer:

→  The agent never calls a taxi home.

**3. `always(¬available_taxi(home))`**

→  "*If for some reasons, the user’s desire is not to see any taxi around his home*" (p.566)"

Preference with asprin: 

In [68]:
%%file travel/pref.lp

#optimize(p3).

% (P3) always(¬available_taxi(home)) -> p. 566.
#preference(p3,more(cardinality)){
    and(initially, always(negation(holds(available_taxi(home)))))
}.

Overwriting travel/pref.lp


In [69]:
!clingo travel/travel.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=2 0 

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(walk(home,school)),2) (occurs(rent_car(home)),1)
OPTIMUM FOUND
Answer: 2
(occurs(walk(home,school)),2) (occurs(buy_ticket(airport1,airport2)),1)
OPTIMUM FOUND *
Answer: 3
(occurs(walk(home,school)),2) (occurs(rent_car(coffeeshop)),1)
OPTIMUM FOUND *
Answer: 4
(occurs(walk(home,school)),2) (occurs(buy_ticket(school,home)),1)
OPTIMUM FOUND *
Answer: 5
(occurs(walk(home,school)),2) (occurs(rent_car(school)),1)
OPTIMUM FOUND *
Answer: 6
(occurs(walk(home,school)),2) (occurs(call_taxi(coffeeshop)),1)
OPTIMUM FOUND *
Answer: 7
(occurs(walk(home,school)),2) (occurs(call_taxi(airport1)),1)
OPTIMUM FOUND *
Answer: 8
(occurs(walk(home,school)),2) (occurs(call_taxi(school)),1)
OPTIMUM FOUND *
Answer: 9
(occurs(walk(home,school)),2) (occurs(rent_car(airport1)),1)
OPTIMUM FOUND *
Answer: 10
(occurs(walk(home,school)),2) (occurs(call_taxi(airport2)),1)
OPTIMUM FOUND *
Answer: 11
(occurs(walk(home,school)),2)
OPTIMUM FOUND *
Answer:

→  This returns the same models as before (no taxi is available at the home if it is not called).

**4. `goal(has_coffee)`** 

→  "*Let us enrich the action theory of Example 2 with an action called buy_coffee, which allows one to have coffee, i.e, the fluent has coffee becomes true. The coffee is not free, i.e., the agent will have to pay some money if he buys coffee. This action can only be executed at the nearby Starbucks shop. [...] Any plan satisfying this preference requires the agent to stop at the Starbucks shop before going to school. For example, while s0 walk(home, school) s1, where s0 and s1 denote the initial state (the agent is at home) and the final state (the agent is at school), respectively, is a valid trajectory for the agent to achieve his goal, this is not a most preferred trajectory; instead, the agent has to go to the Starbucks shop, buy the coffee, and then go to school. Besides the action of buy coffee that is needed for him to get the coffee, the most preferred trajectory requires the action of going to the coffee shop, which is not necessary if he does not have the preference of having the coffee*" (p.568-569)

We already altered `travel/travel.lp` and added: 
    
`action(buy_coffee).`

`eff(buy_coffee, has_coffee, true) :- action(buy_coffee).`

`prec(buy_coffee, has_money, true) :- action(buy_coffee).`

`prec(buy_coffee, at(coffeeshop), true) :- action(buy_coffee).`

`wnext(executable(buy_coffee)) :- holds(at(coffeeshop)), holds(has_money).`

We can encode the preference to have coffee as:

In `travel/auxiliary.lp`: 
    
`p4_aux :- finally, holds(has_coffee).`

In [77]:
%%file travel/pref.lp

#optimize(p4).

% (P4) goal(has_coffee). -> p. 569
#preference(p4,more(cardinality)){
    eventually(p4_aux)
}.

Overwriting travel/pref.lp


In [79]:
!clingo travel/travel.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=3 0 

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(walk(home,school)),3) (occurs(buy_ticket(home,airport1)),2) (occurs(buy_ticket(airport2,home)),1)
Answer: 2
(occurs(buy_coffee),2) (occurs(walk(coffeeshop,school)),3) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND
Answer: 3
(occurs(buy_coffee),2) (occurs(walk(home,coffeeshop)),1) (occurs(bus(coffeeshop,school)),3)
OPTIMUM FOUND *
Answer: 4
(occurs(buy_coffee),2) (occurs(walk(home,coffeeshop)),1) (occurs(walk(coffeeshop,school)),3)
OPTIMUM FOUND *
Answer: 5
(occurs(buy_coffee),2) (occurs(bus(home,coffeeshop)),1) (occurs(bus(coffeeshop,school)),3)
OPTIMUM FOUND *
Answer: 6
(occurs(buy_coffee),2) (occurs(walk(coffeeshop,school)),3) (occurs(bus(home,coffeeshop)),1)
OPTIMUM FOUND *
Answer: 7
(occurs(buy_coffee),2) (occurs(bus(coffeeshop,school)),3) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND *

Models       : 7
  Optimum    : yes
  Optimal    : 6
Calls        : 5
Time         : 2.845s (Solving: 0.07s 1st Model: 0.06

→ Only those trajectories containing `occurs(buy(coffee))` are optimal. 

### Atomic Preferences 

Important Definitions (p. 569):

### Desire Types in Temporal Preferences

- **Strong Desire**:  
  Given the basic desire formulae $ϕ_1$, $ϕ_2$, the preference $ϕ_1 < ϕ_2$ denotes  
  $ϕ_1 \land \lnot ϕ_2$.

- **Weak Desire**:  
  Given the basic desire formulae $ϕ_1$, $ϕ_2$, the preference $ϕ_1 <^w ϕ_2$ denotes  
  $ϕ_1 \lor \lnot ϕ_2$.

- **Enabled Desire**:  
  Given two actions $a_1$, $a_2$, the preference $a_1 <^e a_2$ denotes:

$$
(\text{executable}(a_1) \land \text{executable}(a_2)) \Rightarrow (\text{occ}(a_1) < \text{occ}(a_2))
$$

where:

$$
\text{executable}(a) = \bigvee_{a \text{ executable_if } p_1, \dots, p_k} p_1 \land \dots \land p_k
$$


→ "*Let us continue with our travel domain. Again, let us assume that the agent is at home and he wants to go to school. To simplify the representation, we will write bus, taxi, drive, and walk to say that the agent takes the bus, taxi, drive, or walk to school, respectively. The following is a desire expressing that the agent prefers to get the fastest possible way to go to school (assume that both driving and taking the bus require about the same amount of time to reach the school:*" (p. 570)

**5. `time = always(taxi <e (drive ∨ bus) <e walk)`** 

This preference can be translated with a single temporal formula, considering the definition for **enabled desire** above:

In `travel/auxiliary.lp`: 
    
`% (time) time = always(taxi <e (drive|bus) <e walk) -> p. 570`

`always(time_aux) :- initially, 
always(and(if(and(executable(take_taxi), or(executable(drive), executable(bus))), and(occurs(take_taxi), negation(or(occurs(drive), 
occurs(bus))))), if(and(or(executable(drive), executable(bus)), executable(walk)), and(or(occurs(drive), occurs(bus)), negation(occurs(walk)))))).`

This formula represents the intersection of these two parts:
    
**(1) `taxi <e (drive ∨ bus)`** : `if(and(executable(take_taxi), or(executable(drive), executable(bus))), and(occurs(take_taxi), negation(or(occurs(drive), occurs(bus)))))`

 → If both taking a taxi and driving or taking a bus is executable, to satisfy the formula, the agent has to take the taxi and not the car or the bus. The formula is also satisfied if taking the taxi or driving and taking the bus is not executable. 
    
**(2) `(drive|bus) <e walk)`**: `if(and(or(executable(drive), executable(bus)), executable(walk)), and(or(occurs(drive), occurs(bus)), negation(occurs(walk)))))`

 → If both driving or taking the bus and walking is executable, to satisfy the formula, the agent has to drive or take the bus, not walk. The formula is also satisfied if driving or taking the bus or walking is not executable.
 
 → For the whole formula (see above, always(time_aux)) to be satisfied, (1) and (2) must be true since it is an intersection of both. 

In [86]:
%%file travel/pref.lp

#optimize(time).

%(time) time = always(taxi <e (drive|bus) <e walk) -> p. 570
#preference(time, more(cardinality)){
    and(initially, always(time_aux))
}.

Overwriting travel/pref.lp


In [87]:
!clingo travel/travel.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=2 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(walk(home,school)),2) (occurs(rent_car(home)),1)
Answer: 2
(occurs(bus(coffeeshop,school)),2) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND
Answer: 3
(occurs(bus(home,coffeeshop)),1) (occurs(bus(coffeeshop,school)),2)
OPTIMUM FOUND *

Models       : 3
  Optimum    : yes
  Optimal    : 2
Calls        : 5
Time         : 2.538s (Solving: 0.02s 1st Model: 0.02s Unsat: 0.00s)
CPU Time     : 2.297s


→ Taking the bus and or the car is preferred over walking. Note that taking the taxi is not part of a preferred trajectory since a taxi has to be called first, which would not satisfy the formula: Since all, taking a bus, driving, and walking are executable, the agent has to take a bus or drive (2). Since taxi is not (yet) executable, (1) is satisfied anyways. 

**6. `cost = always(walk <e (drive ∨ bus) <e taxi)`** 

→ "*On the other hand, when the agent is not in a hurry, he/she prefers to get the cheaper way to go to school (assume that driving and taking the bus cost about the same amount of money)*" (p.571)

We can again express this with a temporal formula in ASP, parallel to preference 5.: 

In `travel/auxiliary.lp`: 
    
`% (cost) cost = always(walk <e (drive|bus) <e taxi) -> p. 571
always(cost_aux) :- initially, 
always(and(if(and(executable(walk), or(executable(drive), executable(bus))),
and(occurs(walk), negation(or(occurs(drive), occurs(bus))))), if(and(or(executable(drive), executable(bus)), executable(take_taxi)), and(or(occurs(drive), occurs(bus)), negation(occurs(take_taxi)))))).`

In [88]:
%%file travel/pref.lp

#optimize(cost).

% (cost) cost = always(walk <e (drive|bus) <e taxi) -> p. 571
#preference(cost, more(cardinality)){
    and(initially, always(cost_aux))
}.

Overwriting travel/pref.lp


In [89]:
!clingo travel/travel.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=1 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(walk(home,school)),1)
OPTIMUM FOUND

Models       : 1
  Optimum    : yes
  Optimal    : 1
Calls        : 4
Time         : 2.389s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 2.188s


 → Walking is the cheapest option. 

**7. `comfort = always(flight <e (drive ∨ bus) <e walk)`** 

This is, again, parallel to 5 and 6: 

In `travel/auxiliary.lp`: 
    
`% (comfort) comfort = always(flight <e (drive|bus) <e walk) -> p. 573`

`always(comfort_aux) :- initially, 
always(and(if(and(executable(flight), or(executable(drive), executable(bus))), and(occurs(flight), negation(or(occurs(drive), occurs(bus))))), if(and(or(executable(drive), executable(bus)),
executable(walk)), and(or(occurs(drive), occurs(bus)), negation(occurs(walk)))))).`

In [90]:
%%file travel/pref.lp

#optimize(comfort).

% (comfort) comfort = always(flight <e (drive|bus) <e walk) -> p. 573
#preference(comfort, more(cardinality)){
    and(initially, always(comfort_aux))
}.


Overwriting travel/pref.lp


In [91]:
!clingo travel/travel.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=2 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(walk(home,school)),2) (occurs(rent_car(home)),1)
Answer: 2
(occurs(bus(coffeeshop,school)),2) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND
Answer: 3
(occurs(bus(home,coffeeshop)),1) (occurs(bus(coffeeshop,school)),2)
OPTIMUM FOUND *

Models       : 3
  Optimum    : yes
  Optimal    : 2
Calls        : 5
Time         : 2.373s (Solving: 0.03s 1st Model: 0.02s Unsat: 0.00s)
CPU Time     : 2.281s


→ Again, taking the bus and or the car is preferred over walking. Note that taking a flight is **not** part of a preferred trajectory since tickets have to be bought first, which would not satisfy the second part of the temporal formula (`(drive|bus) <e walk`). 

**8. `safety = always(walk <e flight <e (drive ∨ bus)).`** 

In `travel/auxiliary.lp`: 
    
`% (safety) safety = always(walk <e flight <e (drive | bus)). -> p. 573`

`always(safety_aux) :- initially, 
always(and(if(and(executable(walk), executable(flight)), and(occurs(walk), negation(flight))), if(and(executable(flight), or(executable(drive), executable(bus))), and(occurs(flight), negation(or(occurs(drive), occurs(bus))))))).`

In [92]:
%%file travel/pref.lp

#optimize(safety).

% (safety) safety = always(walk <e flight <e (drive | bus)). -> p. 573
#preference(safety, more(cardinality)){
    and(initially, always(safety_aux))
}.


Overwriting travel/pref.lp


In [93]:
!clingo travel/travel.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=1 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(walk(home,school)),1)
OPTIMUM FOUND
Answer: 2
(occurs(bus(home,school)),1)
OPTIMUM FOUND *
Answer: 3
(occurs(drive(home,school)),1)
OPTIMUM FOUND *

Models       : 3
  Optimum    : yes
  Optimal    : 3
Calls        : 4
Time         : 2.395s (Solving: 0.01s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 2.188s


→ Since flight is never executable in trajectories with `horizon=1`, the temporal formula is always true, meaning all models are optimal.

In [94]:
!clingo travel/travel.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=3 20

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(walk(home,school)),3) (occurs(buy_ticket(home,airport1)),2) (occurs(buy_ticket(airport2,home)),1)
OPTIMUM FOUND
Answer: 2
(occurs(bus(home,school)),3) (occurs(bus(coffeeshop,home)),2) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND *
Answer: 3
(occurs(walk(coffeeshop,home)),2) (occurs(bus(home,school)),3) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND *
Answer: 4
(occurs(walk(home,school)),3) (occurs(walk(coffeeshop,home)),2) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND *
Answer: 5
(occurs(bus(coffeeshop,school)),2) (occurs(drive(home,coffeeshop)),1) (occurs(stop),3)
OPTIMUM FOUND *
Answer: 6
(occurs(walk(home,school)),3) (occurs(bus(coffeeshop,home)),2) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND *
Answer: 7
(occurs(walk(coffeeshop,school)),2) (occurs(drive(home,coffeeshop)),1) (occurs(stop),3)
OPTIMUM FOUND *
Answer: 8
(occurs(walk(coffeeshop,school)),3) (occurs(call_taxi(coffeeshop)),2) (occurs(drive(ho

→ Here, we see a caveat of the Enabled Desire formula: One would expect for the optimal trajectory to only contain walking. But this is not the case. The reason is presented in the following two scenarios: 

1) The agent never buys a plane ticket: Flying is never executable, both parts of the formula `walk <e flight` and  `flight <e (drive | bus)` are always true (since the antecedent is always false), all models are preferred since they satisfy the temporal formula. 

2) The agent buys a plane ticket: Flying is executable, leading to all actions to be executable, leading to a contradiction since both `walk ∧ ¬flight` and `flight ∧ ¬(drive | bus)` must be true to satisfy the formula. Flight cannot be true and false at the same time, so all models containing buying a plane  ticket are not preferred, they do not satisfy the temporal formula. 

→ **All models not containing buying a plane ticket are preferred**.

### General Preferences

"*Now, consider an agent who has in mind the four basic desires time, cost, comfort, and safety. He can rank these preferences and create different atomic preferences, i.e., different orders among these preferences. Let us assume that he has combined these four desires and produced the following two atomic preferences*": (p.573)

**9. `Ψ1 = comfort ◁ safety`** 

**10. ` Ψ2 = cost ◁ time`** 

We can express **9.**  (that *comfort* is preferred over *safety*) as: 

In `travel/auxiliary.lp`: 
    
`% (comfort) comfort = always(flight <e (drive|bus) <e walk) -> p. 573`

`always(comfort_aux) :- initially, 
always(and(if(and(executable(flight), or(executable(drive), executable(bus))), and(occurs(flight), negation(or(occurs(drive), occurs(bus))))), if(and(or(executable(drive), executable(bus)),
executable(walk)), and(or(occurs(drive), occurs(bus)), negation(occurs(walk)))))).`

`% (safety) safety = always(walk <e flight <e (drive | bus)). -> p. 573`

`always(safety_aux) :- initially, 
always(and(if(and(executable(walk), executable(flight)), and(occurs(walk), negation(flight))), if(and(executable(flight), or(executable(drive), executable(bus))), and(occurs(flight), negation(or(occurs(drive), occurs(bus))))))).`

In [95]:
%%file travel/pref.lp

#optimize(p5).

% (comfort) comfort = always(flight <e (drive|bus) <e walk) -> p. 573
#preference(comfort, more(cardinality)){
    and(initially, always(comfort_aux))
}.

% (safety) safety = always(walk <e flight <e (drive | bus)). -> p. 573
#preference(safety, more(cardinality)){
    and(initially, always(safety_aux))
}.


% comfort > safety -> p. 573
#preference(p5,lexico){1 ::**safety; 2::**comfort}.

Overwriting travel/pref.lp


In [96]:
!clingo travel/travel.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=3 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(walk(home,school)),3) (occurs(buy_ticket(home,airport1)),2) (occurs(buy_ticket(airport2,home)),1)
Answer: 2
(occurs(bus(home,coffeeshop)),1) (occurs(bus(coffeeshop,home)),2) (occurs(drive(home,school)),3)
OPTIMUM FOUND
Answer: 3
(occurs(walk(airport1,home)),2) (occurs(bus(home,school)),3) (occurs(drive(home,airport1)),1)
OPTIMUM FOUND *
Answer: 4
(occurs(bus(home,school)),3) (occurs(bus(home,coffeeshop)),1) (occurs(bus(coffeeshop,home)),2)
OPTIMUM FOUND *
Answer: 5
(occurs(bus(home,school)),3) (occurs(bus(coffeeshop,home)),2) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND *

Models       : 5
  Optimum    : yes
  Optimal    : 4
Calls        : 5
Time         : 2.371s (Solving: 0.05s 1st Model: 0.05s Unsat: 0.00s)
CPU Time     : 2.266s


We can express **10.** (that *cost* is preferred over *time*) as: 

In `travel/auxiliary.lp`: 
    
`% (time) time = always(taxi <e (drive|bus) <e walk) -> p. 570
always(time_aux) :- initially, 
always(and(if(and(executable(take_taxi), or(executable(drive), executable(bus))),and(occurs(take_taxi), negation(or(occurs(drive), occurs(bus))))), if(and(or(executable(drive), executable(bus)), executable(walk)), and(or(occurs(drive), occurs(bus)), negation(occurs(walk)))))).`

`% (cost) cost = always(walk <e (drive|bus) <e taxi) -> p. 571
always(cost_aux) :- initially, 
always(and(if(and(executable(walk), or(executable(drive), executable(bus))),
and(occurs(walk), negation(or(occurs(drive), occurs(bus))))), if(and(or(executable(drive), executable(bus)), executable(take_taxi)), and(or(occurs(drive), occurs(bus)), negation(occurs(take_taxi)))))).`

In [97]:
%%file travel/pref.lp

#optimize(p6).

% (cost) cost = always(walk <e (drive|bus) <e taxi) -> p. 571
#preference(cost, more(cardinality)){
    and(initially, always(cost_aux))
}.

%(time) time = always(taxi <e (drive|bus) <e walk) -> p. 570
#preference(time, more(cardinality)){
    and(initially, always(time_aux))
}.


%  cost > time. -> p. 573
#preference(p6,lexico){1 ::**time; 2::**cost}.

Overwriting travel/pref.lp


In [98]:
!clingo travel/travel.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=3 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(walk(home,school)),3) (occurs(buy_ticket(home,airport1)),2) (occurs(buy_ticket(airport2,home)),1)
Answer: 2
(occurs(walk(airport1,home)),2) (occurs(bus(home,school)),3) (occurs(drive(home,airport1)),1)
Answer: 3
(occurs(walk(home,school)),3) (occurs(walk(home,coffeeshop)),1) (occurs(walk(coffeeshop,home)),2)
OPTIMUM FOUND
Answer: 4
(occurs(walk(home,school)),3) (occurs(walk(home,airport1)),1) (occurs(walk(airport1,home)),2)
OPTIMUM FOUND *

Models       : 4
  Optimum    : yes
  Optimal    : 2
Calls        : 6
Time         : 2.449s (Solving: 0.07s 1st Model: 0.07s Unsat: 0.00s)
CPU Time     : 2.344s


"*Intuitively, $Ψ_1$ is a comparison between level of comfort and safety, while $Ψ_2$ is a comparison between affordability and duration. Suppose that the agent would like to combine $Ψ_1$ and $Ψ_2$ to create a preference stating that he prefers trajectories that are as comfortable as possible and cost as little as possible.*" (p. 573)

 → To combine preferences, we need **General Preferences**.

**Definition 10 (General Preferences)**

A general preference formula is a formula satisfying one of the following conditions:

• An atomic preference Ψ is a general preference;

• If $Ψ_1$, $Ψ_2$ are general preferences, then $Ψ_1$ & $Ψ_2$, $Ψ_1$ | $Ψ_2$, and ! $Ψ_1$ are general
preferences;

• If $Ψ_1$, $Ψ_2$,..., $Ψ_k$ is a collection of general preferences, then $Ψ_1$ ◁ $Ψ_2$ ◁···◁ $Ψ_k$
is a general preference.
--- p. 574

*Let us consider the original action theory presented in Section 2 with the action buy coffee and a user having the goal of being at the school and having coffee. Intuitively, every trajectory achieving the goal of the user would require the action of going to the coffee shop, buying the coffee, and going to the school thereafter. Let us consider the following two preferences (similar to those discussed in Example 4):* (p. 579)

`time = always(occ(buy coffee) ∨ (take taxi <e (drive ∨ bus) <e walk))`

`cost = always(occ(buy coffee) ∨ (walk <e (drive ∨ bus) <e take taxi))`

We now add that having a coffee at the end of the trajectory is a hard constraint instead of a soft constraint: 

In [99]:
%%file travel/travel_coffee.lp 

% FLUENTS
% initially (at(home))
holds(at(home)) :- initially.

% initially (has money)
holds(has_money) :-initially.

% initially (available car(home))
holds(available_car(home)) :- initially.

holds(sCost(0)) :- initially.
holds(sCost(NC)) :-
    prev(holds(sCost(N))),
    occurs(A),
    cost(A, C),
    num(N), num(C), NC = N + C, num(NC).

wnext(neg(sCost(N))) :-
    holds(sCost(N)),
    wnext(holds(sCost(M))), N != M.


% ACTIONS WITH THEIR EXECUTABILITY CONDITIONS (ACTION PRECONDITION AXIOMS)

% walk(l1, l2) causes at(l2) if at(l1), road(l1, l2)
action(walk(X,Y)) :- road(X,Y).
prec(walk(X, Y), at(X), true) :- action(walk(X,Y)).

wnext(executable(walk)) :- holds(at(X)), action(walk(X,Y)).

% bus(l1, l2) causes at(l2) if at(l1), road(l1, l2)
action(bus(X,Y)) :- bus_station(X), bus_station(Y), road(X,Y).
prec(bus(X, Y), at(X), true) :- action(bus(X,Y)).

% bus(l1, l2) executable if has money
prec(bus(X, Y), has_money, true) :- action(bus(X,Y)).

wnext(executable(bus)) :- holds(at(X)), action(bus(X,Y)), holds(has_money).


% flight(l1, l2) causes at(l2) if at(l1), has_ticket(l1, l2)
% flight(l1, l2) executable if connected(l1, l2)
action(flight(X,Y)) :- connected(X,Y).
prec(flight(X, Y), at(X), true) :- action(flight(X,Y)).
prec(flight(X, Y), has_ticket(X, Y), true) :- action(flight(X,Y)).
wnext(executable(flight)) :- action(flight(X,Y)), holds(at(X)), holds(has_ticket(X, Y)).

% take_taxi(l1, l2) causes at(l2) if at(l1), road(l1, l2)
action(take_taxi(X,Y)) :- road(X,Y).
prec(take_taxi(X, Y), at(X), true) :- action(take_taxi(X,Y)).


% take_taxi(l1, l2) executable if available_taxi(l1)
prec(take_taxi(X, Y), available_taxi(X), true) :- action(take_taxi(X,Y)).

wnext(executable(take_taxi)) :- action(take_taxi(X,Y)), holds(available_taxi(X)), holds(at(X)).

% call_taxi(l) causes available_taxi(l) if has_money
action(call_taxi(X)) :- location(X).
prec(call_taxi(X), has_money, true) :- action(call_taxi(X)).
wnext(executable(call_taxi)) :- action(call_taxi(X)), holds(has_money).

% rent_car(l) causes available car(l) if has money
action(rent_car(X)) :- location(X).
prec(rent_car(X), has_money, true) :- action(rent_car(X)).
wnext(executable(rent_car)) :- action(rent_car(X)), holds(has_money).

% flight(l1, l2) executable if connected(l1, l2)
wnext(executable(flight)) :- action(flight(X, Y)), holds(connected(X,Y)).

% 
action(drive(X,Y)) :- road(X,Y).
prec(drive(X, Y), available_car(X), true) :- action(drive(X,Y)).
prec(drive(X, Y), at(X), true) :- action(drive(X,Y)).
wnext(executable(drive)) :- holds(available_car(X)), holds(at(X)), action(drive(X,Y)).


% buy ticket(l1, l2) executable if has_money
action(buy_ticket(X,Y)) :- airport(X), airport(Y), X != Y.
prec(buy_ticket(X, Y), has_money, true) :- action(buy_ticket(X,Y)).
wnext(executable(buy_ticket)) :- holds(has_money), action(buy_ticket(X,Y)).

% add this
action(buy_coffee).
prec(buy_coffee, has_money, true) :- action(buy_coffee).
prec(buy_coffee, at(coffeeshop), true) :- action(buy_coffee).
wnext(executable(buy_coffee)) :- holds(at(coffeeshop)), holds(has_money).

:- prec(A,F,true), occurs(A), not prev(holds(F)).
:- prec(A,F,false), occurs(A), prev(holds(F)).

%executable(A) :- prec(A,F,true), prev(holds(F)).
%executable(A) :- prec(A,F,false), not prev(holds(F)).

% ACTIONS WITH THEIR EFFECTS (EFFECTS)

% walk(l1, l2) causes at(l2) if at(l1), road(l1, l2)
eff(walk(X,Y), at(Y), true) :- action(walk(X,Y)).
eff(walk(X,Y), at(X), false) :- action(walk(X,Y)).

% bus(l1, l2) causes at(l2) if at(l1), road(l1, l2)
eff(bus(X,Y), at(Y), true) :- action(bus(X,Y)).
eff(bus(X,Y), at(X), false) :- action(bus(X,Y)).

% flight(l1, l2) causes at(l2) if at(l1), has ticket(l1, l2)
eff(flight(X,Y), at(Y), true) :- action(flight(X,Y)).
eff(flight(X,Y), at(X), false) :- action(flight(X,Y)).


% take_taxi(l1, l2) causes at(l2) if at(l1), road(l1, l2)
eff(take_taxi(X,Y), at(Y), true) :- action(take_taxi(X,Y)).
eff(take_taxi(X,Y), at(X), false) :- action(take_taxi(X,Y)).

% buy_ticket(l1, l2) causes has_ticket(l1, l2)
eff(buy_ticket(X,Y), has_ticket(X,Y), true) :- action(buy_ticket(X,Y)).

% call_taxi(l) causes available_taxi(l) if has money
eff(call_taxi(X), available_taxi(X), true) :- action(call_taxi(X)).

% rent_car(l) causes available_car(l) if has money
eff(rent_car(X), available_car(X), true) :- action(rent_car(X)).


% add action buy_coffee (page 567)
eff(buy_coffee, has_coffee, true) :- action(buy_coffee).

% ADD THIS?
eff(drive(X,Y), at(Y), true) :- action(drive(X,Y)).
eff(drive(X,Y), at(X), false) :- action(drive(X,Y)).
eff(flight(X,Y), has_ticket(X,Y), false) :- action(flight(X,Y)).
eff(take_taxi(X,Y), available_taxi(X), false) :- action(take_taxi(X,Y)).
eff(drive(X,Y), available_car(X), false) :- action(drive(X,Y)).
eff(drive(X,Y), available_car(Y), true) :- action(drive(X,Y)). % add this?


% Update by effects
holds(F) :- occurs(A), eff(A, F, true).
neg(F) :- occurs(A), eff(A, F, false).

%{occurs(X) : action(X)}1.
{ occurs(A) : action(A), A != stop, A != noop } 1 :- not holds(ended).

% INERTIA
wnext(holds(F)) :- holds(F), not wnext(neg(F)).

% holds(F) :- occurs(A), condeffect(A, X, F, true), holds(G) : condition(A,X,G,true);
%                                                 not holds(G) : condition(A,X,G,false).

% neg(F) :- occurs(A), condeffect(A, X, F, false), holds(G) : condition(A,X,G,true);
%                                                 not holds(G) : condition(A,X,G,false).


% GOAL
% holds(goal_achieved) :-  holds(at(school)).

% for preferences time_coffee, cost_coffee, time_n_cost, time_and_cost, time_v_cost, time_or_cost:
holds(goal_achieved) :- holds(at(school)), holds(has_coffee).
:- finally, not holds(goal_achieved).

% as soon as the goal is achieved, fire stop (once)
wnext(occurs(stop)) :- holds(goal_achieved), not holds(ended).
prec(stop, goal_achieved, true).   % stop executable if goal
wnext(executable(stop)) :- holds(goal_achieved), not holds(ended).


eff(stop, ended,     true).        % stop causes ended

% after ended holds, fire noop every step
wnext(occurs(noop)) :- holds(ended).
prec(noop, ended,    true).        % noop executable if ended
wnext(executable(noop)) :-  holds(ended).

eff(noop, ended, true).        % noop causes ended


#external initially.
#external finally.
#external occurs(stop).
#external wnext(occurs(noop)).
#external holds(ended).
#external eventually(holds(goal_achieved)).
#external eventually(holds(at(school))).
#external holds(has_coffee).
#external prev(holds(sCost(N))) : holds(sCost(N)).
#external wnext(holds(sCost(M))) : holds(sCost(M)).
#external prev(holds(sTime(N))) : holds(sTime(N)).
#external wnext(holds(sTime(M))) : holds(sTime(M)).
#external prev(holds(F)) : prec(A,F,true).
#external wnext(neg(F)) : prec(A,F,true).
#external at(X) : location(X).
#external wnext(neg(F)) : prec(A,F,true).
#external eventually(holds(has_coffee)).


#show show(occurs(X)) : action(X).


Overwriting travel/travel_coffee.lp


**11. `time = always(occ(buy_coffee) ∨ (take_taxi <e (drive ∨ bus) <e walk))`**

This is basically the same formula as **6.**, we just need to express that a coffee can also be bought with `or`: 

In `travel/auxiliary.lp`: 

`% (time_coffee) time_coffee = always(occ(buy_coffee) ∨ (take_taxi <e (drive ∨ bus) <e walk)) -> p. 579`

`always(time_coffee_aux) :- initially, 
always(or(occurs(buy_coffee), and(if(and(executable(take_taxi), or(executable(drive), executable(bus))), and(occurs(take_taxi), negation(or(occurs(drive), occurs(bus))))), if(and(or(executable(drive), executable(bus)), executable(walk)), and(or(occurs(drive), occurs(bus)), negation(occurs(walk))))))).`

In [100]:
%%file travel/pref.lp

#optimize(time_coffee).

% (time_coffee) time_coffee = always(occ(buy_coffee) | (take_taxi <e (drive | bus) <e walk)) -> p. 579
#preference(time_coffee, more(cardinality)){
    and(initially, always(time_coffee_aux))
}.

Overwriting travel/pref.lp


In [101]:
!clingo travel/travel_coffee.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=3 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(buy_coffee),2) (occurs(bus(home,coffeeshop)),1) (occurs(bus(coffeeshop,school)),3)
OPTIMUM FOUND
Answer: 2
(occurs(buy_coffee),2) (occurs(drive(home,coffeeshop)),1) (occurs(drive(coffeeshop,school)),3)
OPTIMUM FOUND *
Answer: 3
(occurs(buy_coffee),2) (occurs(bus(coffeeshop,school)),3) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND *

Models       : 3
  Optimum    : yes
  Optimal    : 3
Calls        : 4
Time         : 2.670s (Solving: 0.05s 1st Model: 0.05s Unsat: 0.00s)
CPU Time     : 2.469s


**12. `cost = always(occ(buy coffee) ∨ (walk <e (drive ∨ bus) <e take taxi))`**

Again, this is basically the same formula as **7.**, we just need to express that a coffee can also be bought with `or`: 

In `travel/auxiliary.lp`: 

`% (cost_coffee) cost_coffee = always(occ(buy_coffee) ∨ (walk <e (drive ∨ bus) <e take taxi)). -> p. 579`

`always(cost_coffee_aux) :- initially,
always(or(occurs(buy_coffee), and(if(and(executable(walk), or(executable(drive), executable(bus))), and(occurs(walk), negation(or(occurs(drive), occurs(bus))))), if(and(or(executable(drive), executable(bus)), executable(take_taxi)), and(or(occurs(drive), occurs(bus)), negation(occurs(take_taxi))))))).`

In [102]:
%%file travel/pref.lp

#optimize(cost_coffee).

% (cost_coffee) cost_coffee = always(occ(buy_coffee) | (walk <e (drive | bus) <e tak_taxi)). -> p. 579
#preference(cost_coffee, more(cardinality)){
    and(initially, always(cost_coffee_aux))
}.

Overwriting travel/pref.lp


In [103]:
!clingo travel/travel_coffee.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=3 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(buy_coffee),2) (occurs(bus(home,coffeeshop)),1) (occurs(bus(coffeeshop,school)),3)
Answer: 2
(occurs(buy_coffee),2) (occurs(walk(home,coffeeshop)),1) (occurs(walk(coffeeshop,school)),3)
OPTIMUM FOUND

Models       : 2
  Optimum    : yes
  Optimal    : 1
Calls        : 5
Time         : 2.314s (Solving: 0.03s 1st Model: 0.03s Unsat: 0.00s)
CPU Time     : 2.219s


**13. `Ψ1 = time ∧ cost`**

**14. `Ψ2 = time & cost.`**

*"Observe that $Ψ_1$ is a basic desire while $Ψ_2$ is a general preference. It is easy to see that there are no trajectories satisfying the preference $Ψ_1$. Thus, every trajectory achieving the goal is a most preferred trajectory w.r.t. $Ψ_1$. At the same time, we can show that for every pair of trajectories α and β, neither α $≺_{\Psi_1}$
 β nor β $≺_{\Psi_2}$ α holds."* (p. 579-580)

We can translate **13.** by combining both time_coffee and cost_coffee preferences into a single temporal formula, connected by an intersection: 

**`time_coffee`**: `or(occurs(buy_coffee), and(if(and(executable(walk), or(executable(drive), executable(bus))), and(occurs(walk), negation(or(occurs(drive), occurs(bus))))), if(and(or(executable(drive), executable(bus)), executable(take_taxi)), and(or(occurs(drive), occurs(bus)), negation(occurs(take_taxi))))))`
    
**`cost_coffee`**: `or(occurs(buy_coffee), and(if(and(executable(take_taxi), or(executable(drive), executable(bus))), and(occurs(take_taxi), negation(or(occurs(drive), occurs(bus))))), if(and(or(executable(drive), executable(bus)), executable(walk)), and(or(occurs(drive), occurs(bus)), negation(occurs(walk))))))`

In `travel/auxiliary.lp`: 

`% time n cost, combine time_coffee and cost_coffee within one formula`

`always(time_n_cost_aux) :- initially,`

`always(and(or(occurs(buy_coffee), and(if(and(executable(walk), or(executable(drive), executable(bus))), and(occurs(walk), negation(or(occurs(drive), occurs(bus))))), if(and(or(executable(drive), executable(bus)), executable(take_taxi)), and(or(occurs(drive), occurs(bus)), negation(occurs(take_taxi)))))),`

`or(occurs(buy_coffee), and(if(and(executable(take_taxi), or(executable(drive), executable(bus))), and(occurs(take_taxi), negation(or(occurs(drive), occurs(bus))))), if(and(or(executable(drive), executable(bus)), executable(walk)), and(or(occurs(drive), occurs(bus)), negation(occurs(walk)))))))).`

In [104]:
%%file travel/pref.lp

#optimize(time_n_cost).

#preference(time_n_cost, more(cardinality)){
    and(initially, always(time_n_cost_aux))
    }.

Overwriting travel/pref.lp


In [105]:
!clingo travel/travel_coffee.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=3 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(buy_coffee),2) (occurs(bus(home,coffeeshop)),1) (occurs(bus(coffeeshop,school)),3)
OPTIMUM FOUND
Answer: 2
(occurs(buy_coffee),2) (occurs(walk(coffeeshop,school)),3) (occurs(bus(home,coffeeshop)),1)
OPTIMUM FOUND *
Answer: 3
(occurs(buy_coffee),2) (occurs(drive(home,coffeeshop)),1) (occurs(drive(coffeeshop,school)),3)
OPTIMUM FOUND *
Answer: 4
(occurs(buy_coffee),2) (occurs(bus(coffeeshop,school)),3) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND *
Answer: 5
(occurs(buy_coffee),2) (occurs(walk(coffeeshop,school)),3) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND *
Answer: 6
(occurs(buy_coffee),2) (occurs(walk(home,coffeeshop)),1) (occurs(bus(coffeeshop,school)),3)
OPTIMUM FOUND *
Answer: 7
(occurs(buy_coffee),2) (occurs(walk(home,coffeeshop)),1) (occurs(walk(coffeeshop,school)),3)
OPTIMUM FOUND *

Models       : 7
  Optimum    : yes
  Optimal    : 7
Calls        : 4
Time         : 2.664s (Solving: 0.06s 1st Model:

We can translate **14.** easily with `asprin` as: 

In [106]:
%%file travel/pref.lp

#optimize(time_and_cost).

% (time_coffee) time_coffee = always(occ(buy_coffee) | (take_taxi <e (drive | bus) <e walk)) -> p. 579
#preference(time_coffee, more(cardinality)){
    and(initially, always(time_coffee_aux))
}.

% add coffee as a goal in travel.lp! (different action theory) -> p. 579
% (cost_coffee) cost_coffee = always(occ(buy_coffee) | (walk <e (drive | bus) <e tak_taxi)). -> p. 579
#preference(cost_coffee, more(cardinality)){
    and(initially, always(cost_coffee_aux))
}.

% Ψ2 = time & cost. -> p. 579
#preference(time_and_cost,and){ **time_coffee; **cost_coffee}.

Overwriting travel/pref.lp


In [107]:
!clingo travel/travel_coffee.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=3 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(buy_coffee),2) (occurs(bus(home,coffeeshop)),1) (occurs(bus(coffeeshop,school)),3)
OPTIMUM FOUND
Answer: 2
(occurs(buy_coffee),2) (occurs(drive(home,coffeeshop)),1) (occurs(drive(coffeeshop,school)),3)
OPTIMUM FOUND *
Answer: 3
(occurs(buy_coffee),2) (occurs(bus(coffeeshop,school)),3) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND *
Answer: 4
(occurs(buy_coffee),2) (occurs(walk(home,coffeeshop)),1) (occurs(walk(coffeeshop,school)),3)
OPTIMUM FOUND
Answer: 5
(occurs(buy_coffee),2) (occurs(walk(coffeeshop,school)),3) (occurs(bus(home,coffeeshop)),1)
OPTIMUM FOUND
Answer: 6
(occurs(buy_coffee),2) (occurs(walk(home,coffeeshop)),1) (occurs(bus(coffeeshop,school)),3)
OPTIMUM FOUND *
Answer: 7
(occurs(buy_coffee),2) (occurs(walk(coffeeshop,school)),3) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND *

Models       : 7
  Optimum    : yes
  Optimal    : 7
Calls        : 10
Time         : 2.445s (Solving: 0.06s 1st Model: 0.

→ For both preferences, we see that all trajectories achieving the goal are preferred, since both time and cost cannot be simultaneously satisfied. 

**15. `Ψ3 = time ∨ cost`**

**16. `Ψ4 = time | cost`**

"Here, $Ψ_3$ is a basic desire while $Ψ_4$
is a general preference. We can see that any trajectory containing the actions *taxi* and *walk* would be most preferred with respect to $Ψ_3$. All of these trajectories are indistinguishable. For example, the trajectory

*α = $s_0$ walk(home, coffee shop) $s_1$ buy coffee $s_2$ walk(coffee shop, school) $s_3$*

and the trajectory

*β = $s_0$ walk(home, coffee shop) $s_1$ buy coffee $s_2$ take taxi(coffee shop, school) $s_3$*

are indistinguishable with respect to $Ψ_3$. On the other hand, we have that $\alpha \prec_{\text{cost}} \beta$
 (the minimal cost action is always used) and $\alpha \approx_{\text{time}} \beta$
(the fastest action is not used every time). This implies that $\alpha \prec_{\Psi_4}$" -- p. 580

We can translate **15.** as (same as **13.** but with `or` instead of `and`): 

In `travel/auxiliary.lp`: 

`% time v cost, combine time_coffee or cost_coffee within one formula`

`always(time_v_cost_aux) :- initially,`  

`always(or(or(occurs(buy_coffee), and(if(and(executable(walk), or(executable(drive), executable(bus))), and(occurs(walk), negation(or(occurs(drive), occurs(bus))))), if(and(or(executable(drive), executable(bus)), executable(take_taxi)), and(or(occurs(drive), occurs(bus)), negation(occurs(take_taxi)))))), `

`or(occurs(buy_coffee), and(if(and(executable(take_taxi), or(executable(drive), executable(bus))), and(occurs(take_taxi), negation(or(occurs(drive), occurs(bus))))), if(and(or(executable(drive), executable(bus)), executable(walk)), and(or(occurs(drive), occurs(bus)), negation(occurs(walk)))))))).`

In [108]:
%%file travel/pref.lp

#optimize(time_v_cost).

% Ψ3 = time u cost -> p. 580
#preference(time_v_cost, more(cardinality)){
    and(initially, always(time_v_cost_aux))
    }.

Overwriting travel/pref.lp


In [109]:
!clingo travel/travel_coffee.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=3 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(buy_coffee),2) (occurs(bus(home,coffeeshop)),1) (occurs(bus(coffeeshop,school)),3)
OPTIMUM FOUND
Answer: 2
(occurs(buy_coffee),2) (occurs(drive(home,coffeeshop)),1) (occurs(drive(coffeeshop,school)),3)
OPTIMUM FOUND *
Answer: 3
(occurs(buy_coffee),2) (occurs(bus(coffeeshop,school)),3) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND *
Answer: 4
(occurs(buy_coffee),2) (occurs(walk(coffeeshop,school)),3) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND *
Answer: 5
(occurs(buy_coffee),2) (occurs(walk(coffeeshop,school)),3) (occurs(bus(home,coffeeshop)),1)
OPTIMUM FOUND *
Answer: 6
(occurs(buy_coffee),2) (occurs(walk(home,coffeeshop)),1) (occurs(walk(coffeeshop,school)),3)
OPTIMUM FOUND *
Answer: 7
(occurs(buy_coffee),2) (occurs(walk(home,coffeeshop)),1) (occurs(bus(coffeeshop,school)),3)
OPTIMUM FOUND *

Models       : 7
  Optimum    : yes
  Optimal    : 7
Calls        : 4
Time         : 2.371s (Solving: 0.05s 1st Model:

→ Since taxi is never executable, either walking or taking the bus/car is preferred. 

We can translate **16.** with `asprin's pareto` as: 

In [110]:
%%file travel/pref.lp

#optimize(time_or_cost).

% (time_coffee) time_coffee = always(occ(buy_coffee) | (take_taxi <e (drive | bus) <e walk)) -> p. 579
#preference(time_coffee, more(cardinality)){
    and(initially, always(time_coffee_aux))
}.

% add coffee as a goal in travel.lp! (different action theory) -> p. 579
% (cost_coffee) cost_coffee = always(occ(buy_coffee) | (walk <e (drive | bus) <e tak_taxi)). -> p. 579
#preference(cost_coffee, more(cardinality)){
    and(initially, always(cost_coffee_aux))
}.

% Ψ4 = time | cost -> p. 580
#preference(time_or_cost,pareto){ **time_coffee; **cost_coffee}.


Overwriting travel/pref.lp


In [111]:
!clingo travel/travel_coffee.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=3 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(buy_coffee),2) (occurs(bus(home,coffeeshop)),1) (occurs(bus(coffeeshop,school)),3)
OPTIMUM FOUND
Answer: 2
(occurs(buy_coffee),2) (occurs(drive(home,coffeeshop)),1) (occurs(drive(coffeeshop,school)),3)
OPTIMUM FOUND *
Answer: 3
(occurs(buy_coffee),2) (occurs(bus(coffeeshop,school)),3) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND *
Answer: 4
(occurs(buy_coffee),2) (occurs(walk(home,coffeeshop)),1) (occurs(walk(coffeeshop,school)),3)
OPTIMUM FOUND

Models       : 4
  Optimum    : yes
  Optimal    : 4
Calls        : 7
Time         : 2.519s (Solving: 0.05s 1st Model: 0.05s Unsat: 0.00s)
CPU Time     : 2.344s


→ Here only trajectories **always** satisfiying the preference `time_coffee` or **always** satisfying the preference `cost_coffee` are preferred (so only walking or only driving/taking the bus), which is why we get a subset of the answer sets from the basic desire before. 

**17. `Ψ5 =! time.`**

This can be translated with `asprin's neg` as: 

In [114]:
%%file travel/pref.lp

#optimize(negated_time).

% (time_coffee) time_coffee = always(occ(buy_coffee) | (take_taxi <e (drive | bus) <e walk)) -> p. 579
#preference(time_coffee, more(cardinality)){
    and(initially, always(time_coffee_aux))
}.

% Ψ5 =! time -> p. 580
#preference(negated_time, neg){
    **time_coffee
    }.

Overwriting travel/pref.lp


In [115]:
!clingo travel/travel_coffee.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=3 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(buy_coffee),2) (occurs(bus(home,coffeeshop)),1) (occurs(bus(coffeeshop,school)),3)
Answer: 2
(occurs(buy_coffee),2) (occurs(walk(coffeeshop,school)),3) (occurs(bus(home,coffeeshop)),1)
OPTIMUM FOUND
Answer: 3
(occurs(buy_coffee),2) (occurs(walk(coffeeshop,school)),3) (occurs(drive(home,coffeeshop)),1)
OPTIMUM FOUND *
Answer: 4
(occurs(buy_coffee),2) (occurs(walk(home,coffeeshop)),1) (occurs(walk(coffeeshop,school)),3)
OPTIMUM FOUND *
Answer: 5
(occurs(buy_coffee),2) (occurs(walk(home,coffeeshop)),1) (occurs(bus(coffeeshop,school)),3)
OPTIMUM FOUND *

Models       : 5
  Optimum    : yes
  Optimal    : 4
Calls        : 5
Time         : 2.841s (Solving: 0.06s 1st Model: 0.06s Unsat: 0.00s)
CPU Time     : 2.562s


→ Every trajectory containing `walk` is not preferred. 

**Preference for shortest trajectory – formula based encoding** (p. 592)

For a bound n given, we can also implement a preference for the shortest trajectory. In our example we will use up to `n=5`:

In `travel/auxiliary.lp`: 

`% shortest, for n = 5`

`sigma(0) :- initially, holds(goal_achieved).`

`sigma(1)  :- initially, not holds(goal_achieved), next(holds(goal_achieved)).`

`sigma(2) :- initially, not holds(goal_achieved), not 
             next(holds(goal_achieved)), next(next(holds(goal_achieved))).`
             
`sigma(3) :- initially, not holds(goal_achieved), not next(holds(goal_achieved)), not next(next(holds(goal_achieved))), 
next(next(next(holds(goal_achieved)))).`
            
`sigma(4) :- initially, not holds(goal_achieved), not next(holds(goal_achieved)), not next(next(holds(goal_achieved))), not next(next(next(holds(goal_achieved)))), next(next(next(next(holds(goal_achieved))))).`
            
`sigma(5) :- initially, not holds(goal_achieved), not next(holds(goal_achieved)), not next(next(holds(goal_achieved))), not next(next(next(holds(goal_achieved)))),
not next(next(next(next(holds(goal_achieved))))), next(next(next(next(next(holds(goal_achieved)))))).`

In [116]:
%%file travel/pref.lp

#optimize(shortest_formula_based).

% for n = 5 (4.4.1)  p. 592
#preference(t(W), more(cardinality)){
    and(initially,sigma(W))} : W = 0..5.

#preference(shortest_formula_based,lexico){
    -W :: **t(W) : W = 0..5
    }.

Overwriting travel/pref.lp


In [117]:
!clingo travel/travel_coffee.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=5 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(buy_coffee),4) (occurs(walk(home,coffeeshop)),3) (occurs(walk(coffeeshop,school)),5) (occurs(bus(home,school)),1) (occurs(bus(school,home)),2)
Answer: 2
(occurs(buy_coffee),3) (occurs(walk(coffeeshop,school)),4) (occurs(take_taxi(home,coffeeshop)),2) (occurs(call_taxi(home)),1) (occurs(stop),5)
Answer: 3
(occurs(buy_coffee),2) (occurs(walk(coffeeshop,school)),3) (occurs(bus(home,coffeeshop)),1) (occurs(noop),5) (occurs(stop),4)
OPTIMUM FOUND
Answer: 4
(occurs(buy_coffee),2) (occurs(walk(home,coffeeshop)),1) (occurs(walk(coffeeshop,school)),3) (occurs(noop),5) (occurs(stop),4)
OPTIMUM FOUND *
Answer: 5
(occurs(buy_coffee),2) (occurs(walk(home,coffeeshop)),1) (occurs(bus(coffeeshop,school)),3) (occurs(noop),5) (occurs(stop),4)
OPTIMUM FOUND *
Answer: 6
(occurs(buy_coffee),2) (occurs(walk(coffeeshop,school)),3) (occurs(drive(home,coffeeshop)),1) (occurs(noop),5) (occurs(stop),4)
OPTIMUM FOUND *
Answer: 7
(occurs(buy_coff

Although we give `horizon=5`, the goal can be achieved in 3 timesteps and this is also reflected in the preferred paths. 

**Preference for shortest trajectory – action based encoding** (p. 592)

"The formula based encoding $\textit{short}(n, \varphi)$
requires the bound $\textit{n}$ to be given. We now present another encoding that does not require this condition. We introduce two additional fictitious actions *stop* and *noop* and a new fluent *ended*. The action *stop* will be triggered when the goal is achieved; *noop* is used to fill the slot so that we can compare between trajectories; the fluent ended will denote the fact that the goal has been achieved. We add to the action theory the propositions:


*stop* **causes** *ended*

*stop* **executable if** *ϕ*

*noop* **causes** *ended*

*noop* **executable if** *ended*


Furthermore, we add the condition ¬*ended* to the executability condition of every action in ($\textit{D}$, $\textit{I}$) and to the initial state $\textit{I}$. We can encode the condition of shortest length trajectory as follows. Let


*short* = **always**((*stop* ∨ *noop*) $<^e$ ($a_1$ ∨ ... ∨ $a_k$)).


where $a_1$,...,$a_k$ are the actions in the original action theory. Again, we can show that any most preferred trajectory w.r.t. short is a shortest length trajectory satisfying the goal ϕ. Observe the difference between *short(n, ϕ)* and *short*: both are built using temporal connectives but the former uses fluent formula and the latter uses actions. The second one, we believe, is simpler than the first one; however, it requires some modifications to the original action theory." (p. 592)

We now add *stop*, *noop* and *ended* to our action theory and show with a simpler example that this implementation does not work as suggested in the paper:

In [ ]:
%%file travel/test.lp

% FLUENTS

% clingo travel/test.lp travel/auxiliary.lp travel/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=1 0
% ACTIONS WITH THEIR EXECUTABILITY CONDITIONS (ACTION PRECONDITION AXIOMS)

action(do_f).
action(wait).

:- prec(A,F,true), occurs(A), not prev(holds(F)).
:- prec(A,F,false), occurs(A), prev(holds(F)).


% ACTIONS WITH THEIR EFFECTS (EFFECTS)
eff(do_f, f, true) :- action(do_f).
executable(do_f).
executable(wait).

% Update by effects
holds(F) :- occurs(A), eff(A, F, true).

{ occurs(A) : action(A), A != stop, A != noop } 1 :- not holds(ended).

% INERTIA
wnext(holds(F)) :- holds(F), not wnext(neg(F)).


% GOAL
holds(goal_achieved) :- holds(f).
:- finally, not holds(goal_achieved).

% as soon as the goal is achieved, fire stop (once)
wnext(occurs(stop)) :- holds(goal_achieved), not holds(ended).
prec(stop, goal_achieved, true).   % stop executable if goal
wnext(executable(stop)) :- holds(goal_achieved), not holds(ended).


eff(stop, ended, true).        % stop causes ended

% after ended holds, fire noop every step
wnext(occurs(noop)) :- holds(ended).
prec(noop, ended,    true).        % noop executable if ended
wnext(executable(noop)) :-  holds(ended).

eff(noop, ended, true).        % noop causes ended



#external initially.
#external finally.
#external occurs(stop).
#external wnext(occurs(noop)).
#external holds(ended).
#external eventually(holds(goal_achieved)).
#external eventually(holds(f)).
#external prev(holds(F)) : prec(A,F,true).
#external wnext(neg(F)) : prec(A,F,true).

#show show(occurs(X)) : action(X).
#show show(holds(F)) : eff(A, F, true).


The temporal formula  `short = always((stop ∨ noop) <e (a1 ∨ ... ∨ ak))`. is translated in `travel/auxiliary.lp` as: 
    
`always(short_aux_test) :- initially, 
always(if(and(or(executable(stop), executable(noop)), or(executable(wait), executable(do_f))), and(or(occurs(stop), occurs(noop)), negation(or(occurs(wait), occurs(do_f)))))).`

 → Translation: If `stop` or `noop` and any other action is executable, `stop` or `noop` shoud be occur, no other action.

In [126]:
%%file travel/pref.lp

#optimize(shortest_action_based).

% for unbound n (4.4.2)-> p. 592
% run with test.lp
% short = always((stop v noop) <e (a1 v ... v ak)).
#preference(shortest_action_based,  more(cardinality)) {
    and(initially, always(short_aux_test))
}.


Overwriting travel/pref.lp


In [127]:
!clingo travel/test.lp travel/auxiliary.lp travel/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=1 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(holds(ended),1) (holds(f),1) (holds(f),0) (occurs(do_f),0) (occurs(stop),1)
OPTIMUM FOUND
Answer: 2
(holds(f),1) (occurs(do_f),1)
OPTIMUM FOUND *
Answer: 3
(holds(f),1) (occurs(do_f),1) (occurs(wait),0)
OPTIMUM FOUND *

Models       : 3
  Optimum    : yes
  Optimal    : 3
Calls        : 4
Time         : 0.289s (Solving: 0.01s 1st Model: 0.01s Unsat: 0.00s)
CPU Time     : 0.281s


→ We see that all trajectories are preferred although the goal can be achieved in the first step `((holds(ended),1) (holds(f),1) (holds(f),0) (occurs(do_f),0) (occurs(stop),1))`. This is because, if the action that achieves this goal is not executed, neither `stop` nor `noop` is executable, leading for the formula to be true in these cases as well. Thus, all models are optimal

**Cheapest Plan** (p. 592)

We can associate costs to each action and determine trajectories that have minimal cost by adding up values of all actions and then minimizing that number (see p. 593). We simply added this to `travel/travel.lp`:

`holds(sCost(0)) :- initially. `
`holds(sCost(NC)) :-
    prev(holds(sCost(N))),
    occurs(A),
    cost(A, C),
    num(N), num(C), NC = N + C, num(NC).`

`wnext(neg(sCost(N))) :-
    holds(sCost(N)),
    wnext(holds(sCost(M))), N != M.`

.... and this* into `travel/instances.lp`:

`% Costs`
`cost(walk(X,Y), 0) :- action(walk(X,Y)).
cost(bus(X,Y), 2) :- action(bus(X,Y)).
cost(flight(X,Y), 0) :- action(flight(X,Y)).
cost(take_taxi(X,Y), 3) :- action(take_taxi(X,Y)).
cost(call_taxi(X), 0) :- action(call_taxi(X)).
cost(rent_car(X), 0) :- action(rent_car(X)).
cost(drive(X,Y), 2) :- action(drive(X,Y)).
cost(buy_ticket(X,Y), 10) :- action(buy_ticket(X,Y)).
cost(buy_coffee, 1) :- action(buy_coffee).
cost(noop, 0).
cost(stop, 0).`

`#const maxCost=100.
num(0..maxCost).`

\*Note: Costs for actions are not given in the paper, we assigned these values.

In [128]:
%%file travel/pref.lp

#const maxCost=100.

#optimize(cheapest).

% cheapest trajectory -> p. 593
#preference(c(W), more(cardinality)){
    eventually(cost_overall(W))}: W = 0..maxCost.

#preference(cheapest,lexico){
    -W :: **c(W) : W = 0..maxCost
    }.

Overwriting travel/pref.lp


In [129]:
!clingo travel/travel_coffee.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=3 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(buy_coffee),2) (occurs(bus(home,coffeeshop)),1) (occurs(bus(coffeeshop,school)),3)
Answer: 2
(occurs(buy_coffee),2) (occurs(walk(coffeeshop,school)),3) (occurs(bus(home,coffeeshop)),1)
Answer: 3
(occurs(buy_coffee),2) (occurs(walk(home,coffeeshop)),1) (occurs(walk(coffeeshop,school)),3)
OPTIMUM FOUND

Models       : 3
  Optimum    : yes
  Optimal    : 1
Calls        : 6
Time         : 2.810s (Solving: 0.06s 1st Model: 0.05s Unsat: 0.00s)
CPU Time     : 2.562s


→ The cheapest plan contains only walking (note that the agent still needs to get a coffee to achieve the goal).

**Additional implementation--Weak Desire**

When implementing the "enabled desires" such as `time = always(taxi <e (drive ∨ bus) <e walk)` (p.570) we noticed some behaviours that may be undesirable:

1) Since taking a taxi is never executable without calling a taxi first (at least in this problem setting), a taxi is never called. If we called a taxi at a time step instead of just taking the next best option, i.e. taking the car or a bus, the formula would not be satisfied, leading to a non-optimal model. 

2) If **all** actions are executable at the same time, there is a contradiction in the formula since a. Since *taxi* and *(drive v bus)* is executable, *taxi* must be true and *(drive v bus)* must be false and since b. Since *drive v bus* and *walk* is executable, *(drive v bus)* must be true and *walk* must be false. So *(drive v bus)* would both be true and false, leading to this contradiction, never satisfying the formula. 

In order to avoid these drawbacks, one could implement these desires with the formula for a weak basic desire instead of an enabled desire. We present this approach for `time = always(taxi <w (drive ∨ bus) <w walk)`:

**Time (Weak Desire)**

In `travel/auxiliary.lp`:

`% (time weak) time = always(taxi <w (drive|bus) <w walk) -> p. 570`

`always(time_weak_aux) :- initially, always(and(or(occurs(take_taxi), negation(or(occurs(drive), occurs(bus)))), or(or(occurs(drive), occurs(bus)), negation(occurs(walk))))).`

In [130]:
%%file travel/pref.lp
#optimize(time_weak).

#preference(time_weak, more(cardinality)){
    and(initially, always(time_weak_aux))
}.

Overwriting travel/pref.lp


In [131]:
!clingo travel/travel.lp travel/instances.lp travel/auxiliary.lp travel/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp travel/pref.lp meta-telingo-extension.lp -c horizon=2 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(walk(home,school)),2) (occurs(rent_car(home)),1)
Answer: 2
(occurs(take_taxi(home,school)),2) (occurs(call_taxi(home)),1)
OPTIMUM FOUND
Answer: 3
(occurs(flight(home,school)),2) (occurs(buy_ticket(home,school)),1)
OPTIMUM FOUND *

Models       : 3
  Optimum    : yes
  Optimal    : 2
Calls        : 5
Time         : 2.958s (Solving: 0.03s 1st Model: 0.02s Unsat: 0.01s)
CPU Time     : 2.625s


→ Now, instead of taking the bus or car immediately, the agent can call a taxi (or buy a plane ticket) and fulfill the preference.

## Monkey Problem 🐒🍌🍫🥥

Modelling after paper: **Son, T. C., & Pontelli, E. (2006). Planning with preferences using logic programming. Theory and Practice of Logic Programming, 6(5), 559-607.** https://www.cambridge.org/core/journals/theory-and-practice-of-logic-programming/article/planning-with-preferences-using-logic-programming/441D04E11B5B0008367591DA813382A7

*"Let us consider the monkey-and-banana example as formulated in Delgrande et al.
(2004). The world includes the following entities: a monkey, a banana hanging from
the ceiling, a coconut on the floor, and a chocolate bar inside a closed drawer.
Initially, all the entities are in different locations in a room. The room includes also
a box that can be pushed and climbed on to reach the ceiling and grab the banana.
The goal is to get the chocolate as well as at least one of the banana or the coconut." 
The domain description includes the following fluents: (p. 594)*

Again, we can directly translate the problem into ASP: 

• *location(Entity, Location) denoting the current Location of Entity; the domain
of Entity is {monkey, banana, coconut, drawer, box} and the domain of Location
is {1,..., 5} (denoting 5 different positions in the room).*

... can be translated as ...

In [132]:
%%file monkey/instances.lp

% Entities
entity(monkey; banana; coconut; drawer; box).

% Locations
location(1..5).

Overwriting monkey/instances.lp


*"The action theory provides actions to walk in the room, move the box, climb on and off the box, grab objects, and open drawers. The goal considered here is expressed by the fluent formula: `hasChocolate ∧ (hasCoconut ∨ hasBanana)`"*

The monkey problem can thus be translated (as we've seen before) by the help of fluents and action items: 

In [133]:
%%file monkey/monkey.lp
% FLUENTS

% 2: ceiling, 3: floor, 4: drawer
holds(at(monkey, 1)) :- initially.
holds(at(banana, 2)) :- initially.
holds(at(coconut, 3)) :- initially.
holds(at(drawer, 4)) :- initially.
holds(at(chocolate, 4)) :- initially.
holds(at(box, 5)) :- initially.

% ACTIONS WITH THEIR EXECUTABILITY CONDITIONS (ACTION PRECONDITION AXIOMS)
action(walk(X,Y)) :- location(X), location(Y), X != Y.
prec(walk(X,Y), at(monkey, Y), false) :- action(walk(X,Y)).
prec(walk(X,Y), at(monkey, X), true) :- action(walk(X,Y)).
prec(walk(X,Y), onBox, false) :- action(walk(X,Y)).

action(push(X,Y)) :- location(X), location(Y), X!=Y.
prec(push(X,Y), at(monkey,X), true) :- action(push(X,Y)).
prec(push(X,Y), at(box,X), true) :- action(push(X,Y)).
prec(push(X,Y), onBox, false) :- action(push(X,Y)).

action(climb).
prec(climb, onBox, false) :- action(climb).
prec(climb, at(monkey,X), true) :- action(climb), holds(at(box,X)).

action(descend).
prec(descend, onBox, true) :- action(descend).

action(grasp).
prec(grasp, onBox, true) :- action(grasp).
prec(grasp, at(monkey, X), true) : holds(at(box,X)) :- action(grasp).
prec(grasp, at(banana, X), true) : holds(at(box,X)) :- action(grasp).

action(open).
prec(open, drawerOpen, false) :- action(open).
prec(open, at(monkey,X), true) : holds(at(drawer,X)) :- action(open).

action(take_chocolate).
prec(take_chocolate, drawerOpen, true) :- action(take_chocolate).
prec(take_chocolate, at(monkey, X), true) : holds(at(chocolate,X)) :- action(take_chocolate).


action(take_coconut).
prec(take_coconut, at(monkey, X) , true) : holds(at(coconut,X)) :- action(take_coconut).

:- prec(A,F,true), occurs(A), not prev(holds(F)).
:- prec(A,F,false), occurs(A), prev(holds(F)).

% ACTIONS WITH THEIR EFFECTS (EFFECTS)
eff(walk(X,Y), at(monkey, Y), true) :- action(walk(X,Y)).
eff(walk(X,Y), at(monkey, X), false) :- action(walk(X,Y)).

eff(push(X,Y), at(monkey, Y), true) :- action(push(X,Y)).
eff(push(X,Y), at(box, Y), true) :- action(push(X,Y)).

eff(push(X,Y), at(monkey, X), false) :- action(push(X,Y)).
eff(push(X,Y), at(box, X), false) :- action(push(X,Y)).

eff(grasp, hasBanana, true) :- action(grasp).

eff(climb, onBox, true) :- action(climb).

eff(descend, onBox, false) :- action(descend).

eff(open, drawerOpen, true) :- action(open).

eff(take_coconut, hasCoconut, true) :- action(take_coconut).

eff(take_chocolate, hasChocolate, true) :- action(take_chocolate).

% Update by effects
holds(F) :- occurs(A), eff(A, F, true).
neg(F) :- occurs(A), eff(A, F, false).

{ occurs(A) : action(A), A != stop, A != noop } 1 :- not holds(ended).

% Inertia
wnext(holds(F)) :- holds(F), not wnext(neg(F)).

% GOAL
holds(goal_achieved) :- holds(hasCoconut), holds(hasChocolate).
holds(goal_achieved) :- holds(hasBanana), holds(hasChocolate).
    
:- not eventually(holds(goal_achieved)).


% as soon as the goal is achieved, fire stop (once)
wnext(occurs(stop)) :- holds(goal_achieved), not holds(ended).
prec(stop, goal_achieved, true).   % stop executable if goal
wnext(executable(stop)) :- holds(goal_achieved), not holds(ended).


eff(stop, ended,     true).        % stop causes ended

% after ended holds, fire noop every step
wnext(occurs(noop)) :- holds(ended).
prec(noop, ended,    true).        % noop executable if ended
wnext(executable(noop)) :-  holds(ended).

eff(noop, ended, true).        % noop causes ended



#external initially.
#external eventually(holds(goal_achieved)).
#external eventually(occurs(noop)).
#external eventually(holds(ended)).
#external occurs(stop).
#external finally.
#external and(holds(hasCoconut), or(holds(hasChocolate), holds(hasBanana))).
#external prev(holds(F)) : prec(A,F,true).
#external wnext(neg(F)) : prec(A,F,true).
#external wnext(neg(F)) : prec(A,F,true).

#show show(occurs(X)) : action(X).

Overwriting monkey/monkey.lp


More specifically, the goal can be expressed with the temporal formula: 
    `and(holds(hasCoconut), or(holds(hasChocolate), holds(hasBanana)))`

The program is satifsied with at least 5 time steps:

In [134]:
!clingo monkey/monkey.lp monkey/instances.lp  0 --output=reify | clingo - meta-telingo.lp -c horizon=5 0

clingo version 5.4.0
Reading from - ...
Solving...
Answer: 1
(occurs(take_coconut),5) (occurs(walk(4,3)),4) (occurs(take_chocolate),3) (occurs(open),2) (occurs(walk(1,4)),1)
Answer: 2
(occurs(take_chocolate),5) (occurs(open),4) (occurs(walk(3,4)),3) (occurs(take_coconut),2) (occurs(walk(1,3)),1)
SATISFIABLE

Models       : 2
Calls        : 1
Time         : 0.111s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.109s


But we also see that we quickly obtain a large number of answer sets: 

In [135]:
!clingo monkey/monkey.lp monkey/instances.lp  0 --output=reify | clingo - meta-telingo.lp -c horizon=6 0

clingo version 5.4.0
Reading from - ...
Solving...
Answer: 1
(occurs(take_coconut),6) (occurs(walk(4,3)),5) (occurs(take_chocolate),4) (occurs(open),3) (occurs(push(5,4)),2) (occurs(walk(1,5)),1)
Answer: 2
(occurs(take_coconut),6) (occurs(push(4,3)),5) (occurs(take_chocolate),4) (occurs(open),3) (occurs(push(5,4)),2) (occurs(walk(1,5)),1)
Answer: 3
(occurs(take_chocolate),5) (occurs(open),4) (occurs(walk(3,4)),3) (occurs(take_coconut),2) (occurs(walk(1,3)),1)
Answer: 4
(occurs(take_coconut),6) (occurs(walk(4,3)),5) (occurs(take_chocolate),4) (occurs(open),3) (occurs(walk(1,4)),1)
Answer: 5
(occurs(take_coconut),6) (occurs(walk(4,3)),5) (occurs(take_chocolate),4) (occurs(open),3) (occurs(walk(3,4)),2) (occurs(walk(1,3)),1)
Answer: 6
(occurs(take_coconut),6) (occurs(walk(4,3)),5) (occurs(take_chocolate),4) (occurs(open),3) (occurs(walk(1,4)),2)
Answer: 7
(occurs(take_coconut),6) (occurs(walk(4,3)),5) (occurs(take_chocolate),4) (occurs(open),3) (occurs(walk(2,4)),2) (occurs(walk(1,2)),1)


Thus, we will try to satisfy some preferences. 

1) "*The preference discussed in Delgrande et al. Delgrande et al. (2004) is that bananas
are preferred over coconuts – i.e., hasCoconut ≤c hasBanana – and in our framework
it can be expressed as*

`eventually(hasBanana)◁ eventually(hasCoconut)`"

We can represent this lexico preference in monkey/pref.lp as: 

`% (P1) eventually(hasBanana) > eventually(hasCoconut). -> p. 594`

`#preference(hasBanana, more(cardinality)){
    eventually(holds(hasBanana))}.`

`#preference(hasChocolate, more(cardinality)){
    eventually(holds(hasChocolate))}.`

`#preference(p1,lexico){1 ::**hasChocolate; 2::**hasBanana}.`


2) "*Let us continue Example 8 by removing the choice preference and assuming instead the temporal preference hasBanana ≤t hasChocolate – i.e., the banana should be obtained before the chocolate. The corresponding encoding in our language is*

`eventually(hasBanana ∧ eventually(hasChocolate)) ∧ until(¬hasChocolate, hasBanana)`"

`% (P2) eventually(hasBanana n eventually(hasChocolate)) n until(not hasChocolate, hasBanana) -> p. 595`

`#preference(p2,more(cardinality)){
    and(and(eventually(holds(hasBanana)),
    eventually(holds(hasChocolate))),
    until(negation(holds(hasChocolate)), holds(hasBanana)))
    }.`

We can now test this out by optimizing either with p1 (having a banana is more important than having chocolate):

In [136]:
%%file monkey/pref.lp
% clingo monkey/monkey.lp monkey/instances.lp  0 --output=reify | clingo - meta-telingo.lp -c horizon=2 0 --project
% clingo monkey/monkey.lp monkey/instances.lp monkey/auxiliary.lp monkey/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp monkey/pref.lp meta-telingo-extension.lp -c horizon=2 0

#optimize(p1).

% (P1) eventually(hasBanana) > eventually(hasCoconut). -> p. 594
#preference(hasBanana, more(cardinality)){
    eventually(holds(hasBanana))}.

#preference(hasChocolate, more(cardinality)){
    eventually(holds(hasChocolate))}.

#preference(p1,lexico){1 ::**hasChocolate; 2::**hasBanana}.


% (P2) eventually(hasBanana n eventually(hasChocolate)) n until(not hasChocolate, hasBanana) -> p. 595

#preference(p2,more(cardinality)){
    and(and(eventually(holds(hasBanana)),
    eventually(holds(hasChocolate))),
    until(negation(holds(hasChocolate)), holds(hasBanana)))
    }.


Overwriting monkey/pref.lp


In [138]:
!clingo monkey/monkey.lp monkey/instances.lp monkey/auxiliary.lp monkey/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp monkey/pref.lp meta-telingo-extension.lp -c horizon=7 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(open),2) (occurs(take_chocolate),7) (occurs(take_coconut),5) (occurs(walk(1,4)),1) (occurs(walk(3,4)),6) (occurs(walk(4,5)),3) (occurs(push(5,3)),4)
Answer: 2
(occurs(climb),6) (occurs(grasp),7) (occurs(open),2) (occurs(take_chocolate),3) (occurs(walk(1,4)),1) (occurs(walk(4,5)),4) (occurs(push(5,2)),5)
OPTIMUM FOUND
Answer: 3
(occurs(climb),6) (occurs(grasp),7) (occurs(open),3) (occurs(take_chocolate),4) (occurs(walk(1,5)),1) (occurs(push(4,2)),5) (occurs(push(5,4)),2)
OPTIMUM FOUND *

Models       : 3
  Optimum    : yes
  Optimal    : 2
Calls        : 5
Time         : 0.322s (Solving: 0.02s 1st Model: 0.01s Unsat: 0.01s)
CPU Time     : 0.297s


or with p2 (the banana should be obtained before the chocolate): 

In [139]:
%%file monkey/pref.lp
% clingo monkey/monkey.lp monkey/instances.lp  0 --output=reify | clingo - meta-telingo.lp -c horizon=2 0 --project
% clingo monkey/monkey.lp monkey/instances.lp monkey/auxiliary.lp monkey/pref-externals.lp  0 --output=reify | asprin - meta-telingo.lp monkey/pref.lp meta-telingo-extension.lp -c horizon=2 0

#optimize(p2).

% (P1) eventually(hasBanana) > eventually(hasCoconut). -> p. 594
#preference(hasBanana, more(cardinality)){
    eventually(holds(hasBanana))}.

#preference(hasChocolate, more(cardinality)){
    eventually(holds(hasChocolate))}.

#preference(p1,lexico){1 ::**hasChocolate; 2::**hasBanana}.


% (P2) eventually(hasBanana n eventually(hasChocolate)) n until(not hasChocolate, hasBanana) -> p. 595

#preference(p2,more(cardinality)){
    and(and(eventually(holds(hasBanana)),
    eventually(holds(hasChocolate))),
    until(negation(holds(hasChocolate)), holds(hasBanana)))
    }.


Overwriting monkey/pref.lp


In [140]:
!clingo monkey/monkey.lp monkey/instances.lp monkey/auxiliary.lp monkey/pref-externals.lp 0 --output=reify | \
 asprin - meta-telingo.lp monkey/pref.lp meta-telingo-extension.lp -c horizon=8 0

asprin version 3.1.1
Reading from - ...
Solving...
Answer: 1
(occurs(climb),3) (occurs(descend),5) (occurs(open),4) (occurs(take_chocolate),6) (occurs(take_coconut),8) (occurs(walk(1,5)),1) (occurs(push(4,3)),7) (occurs(push(5,4)),2)
Answer: 2
(occurs(climb),3) (occurs(descend),5) (occurs(grasp),4) (occurs(open),7) (occurs(take_chocolate),8) (occurs(walk(2,4)),6) (occurs(walk(1,5)),1) (occurs(push(5,2)),2)
OPTIMUM FOUND
Answer: 3
(occurs(climb),3) (occurs(descend),5) (occurs(grasp),4) (occurs(open),7) (occurs(take_chocolate),8) (occurs(walk(1,5)),1) (occurs(push(5,2)),2) (occurs(push(2,4)),6)
OPTIMUM FOUND *

Models       : 3
  Optimum    : yes
  Optimal    : 2
Calls        : 5
Time         : 0.341s (Solving: 0.03s 1st Model: 0.03s Unsat: 0.00s)
CPU Time     : 0.328s


Here, we see that the monkey first takes the banana (grasp) and then the takes the chocolate.